# Imports

In [860]:
import pandas as pd
import numpy as np

# Phase 1
> 2022-2024; NA Listed Firms;

## Emissions Data

### Load

In [861]:
""" Read in the CSV """
raw_emissions_df = pd.read_csv(
    "phase1_emissions.csv", 
    dtype={"companyid": "str", "gvkey": "str"}, #identifiers
    parse_dates=["periodenddate"], 
)

raw_emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
0,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,6520204,NaN,Factory Mutual Insurance Company,United States
1,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,24951392,NaN,"Factory Mutual Insurance Company, Asset Manage...",United States
2,11489,2022,2022-12-31,1859.947024,5704.373536,78880.089343,43964734,NaN,"Allendale Mutual Insurance Co, Asset Managemen...",United States
3,11489,2023,2023-12-31,4154.828402,12742.671094,176205.682880,6520204,NaN,Factory Mutual Insurance Company,United States
4,11489,2023,2023-12-31,4154.828402,12742.671094,176205.682880,24951392,NaN,"Factory Mutual Insurance Company, Asset Manage...",United States
...,...,...,...,...,...,...,...,...,...,...
1803283,118526590,2022,2022-12-31,485.540053,1085.798816,1022.548683,111952300,NaN,Coseva Soc Coop,Italy
1803284,118706040,2022,2022-12-31,205.995561,658.395477,1709.587413,1886594015,NaN,Solà Domingo S.A.,Spain
1803285,118918440,2022,2022-07-31,40.124678,26.875831,83.556928,1887792515,NaN,"Nihonsangyo Co.,Ltd.",Japan
1803286,118991426,2022,2022-12-31,59.719416,216.809138,129.375410,1888091056,NaN,CP Management Spólka Z Ograniczona Odpowiedzia...,Poland


### EDA (Can be skipped)

**Inspect the loaded datatypes**

In [679]:
raw_emissions_df.dtypes

institutionid             int64
fiscalyear                int64
periodenddate    datetime64[ns]
di_319413               float64
di_319414               float64
di_319415               float64
companyid                object
gvkey                    object
companyname              object
country                  object
dtype: object

**The company ids are extracted and written out to a file**

In [680]:
""" Utility functions regarding validity, uniqueness, and writing out 
unique and valid ids. """

def keep_valid(data, colname=None):
    """ Only keeps the rows/items from the given dataframe/array-like which 
    have valid values - for the specified column in the case of a dataframe.

    Args:
        data: Dataframe or array-like.
        colname: Name of the dataframe column whose valid values we are using to 
            filter.

    Returns:
        The dataframe/array-like with rows/items that have invalid values, w.r.t 
        the specified column if applicable, filtered out.
    """

    # Array whose values we are interested in
    col = data[colname] if colname is not None else data
    # Values are considered valid as long as they are not NaN
    return data[pd.notnull(col)]


def extract_unique(df, colname):
    """ Extract the unique and non-NaN values from a dataframe column.
    
    Args:
        df: Dataframe.
        colname: Column name w.r.t the dataframe.

    Returns:
        Unique and non-NaN column values.
    """

    return keep_valid(pd.unique(df[colname]))


def write_ids(df, idname, filename):
    """ Writes the unique, non-NaN instances of the indicated identifier, within the 
    indicated dataframe, on separate lines of a new file, whose filename should 
    be specified.

    Args:
        df: Dataframe.
        idname: Column name of the identifier with respect to the dataframe.
        filename: The name to use for the newly created file.
    """
    with open(filename, "w") as fh:
        for idval in extract_unique(df, idname):
            fh.write(f"{idval}\n")

In [4]:
""" Export CIQ company ids """
write_ids(raw_emissions_df, "companyid", "companyids.txt")

**The number of unique company ids and gvkeys are reported for this raw emissions data**

In [681]:
""" Utility function """
def report_unique(df, colname):
    """ Reports unique, non-NaN values of a dataframe column
    
    Args:
        df: Dataframe.
        colname: Column name.
    """

    print(f"Number of unique, non-null values of \'{colname}\': {len(extract_unique(df, colname))}")

In [682]:
print("-- Raw Emissions Data")
report_unique(raw_emissions_df, "institutionid")
report_unique(raw_emissions_df, "companyid")
report_unique(raw_emissions_df, "gvkey")

-- Raw Emissions Data
Number of unique, non-null values of 'institutionid': 1720932
Number of unique, non-null values of 'companyid': 1725359
Number of unique, non-null values of 'gvkey': 24304


**Check whether every gvkey entry has a corresponding company id (non-null -> non-null)**

In [684]:
def is_backed(df, src_colname, ref_colname):
    """ Check if one column is backed by another in a dataframe. This means that 
    when the first column is non-null, the second column cannot be null as 
    otherwise it would be backing the first column.

    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    
    Returns:
        True/False depending on whether the first column is backed by the second
    """

    # check for 0 invalid cases for valid backing
    return (df[src_colname].notnull() & df[ref_colname].isnull()).sum() == 0

def report_backed(df, src_colname, ref_colname):
    """ Reports whether every non-null value of src is backed by a non-null 
    value of ref.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that should be backed.
        ref_colname: Name of backing column.
    """
    print(f"Is \'{src_colname}\' backed by \'{ref_colname}\': {'YES' if is_backed(df, src_colname, ref_colname) else 'NO'}")

In [685]:
print("-- Raw Emissions Data")
report_backed(
    raw_emissions_df, "gvkey", "companyid"
)
report_backed(
    raw_emissions_df, "companyid", "institutionid"
)

-- Raw Emissions Data
Is 'gvkey' backed by 'companyid': YES
Is 'companyid' backed by 'institutionid': YES


**Check for duplicates and null values in the 3 identifiers - companyid, gvkey and institutionid**

In [686]:
""" Utilty functions """
def report_null(df, colname):
    """ Indicate the number of nulls in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """
    print(f"Number of nulls in \'{colname}\': {df[colname].isnull().sum()}")

def report_dup(df, colname):
    """ Indicate the presence of duplicates in the specified column in the 
    given dataframe.
    
    Args:
        df: Dataframe.
        src_colname: Name of column that we are interested in.
    """

    print(f"Are there duplicates in \'{colname}\': {'YES' if df[colname].duplicated().any() else 'NO'}")

In [687]:
print("-- Raw Emissions Data")
print("> Null checks")
report_null(raw_emissions_df, "companyid")
report_null(raw_emissions_df, "institutionid")
report_null(raw_emissions_df, "gvkey")
print("> Dup checks")
report_dup(raw_emissions_df, "companyid")
report_dup(raw_emissions_df, "institutionid")
report_dup(raw_emissions_df, "gvkey")

-- Raw Emissions Data
> Null checks
Number of nulls in 'companyid': 747
Number of nulls in 'institutionid': 0
Number of nulls in 'gvkey': 1769291
> Dup checks
Are there duplicates in 'companyid': YES
Are there duplicates in 'institutionid': YES
Are there duplicates in 'gvkey': YES


**Utilities for numerical value conflicts on grouping by a key. Check for one to one mapping conflicts between identifiers.**

In [688]:
""" Utility functions """

# aggregates series values, NaN if all equal, else string representation with space delimiter
# TODO: assumes no NaN in vals and also the existence of at least 1 element due 
# to the retrieval at index 0.
numer_conflict_aggr = lambda vals: np.nan if (vals.iloc[0] == vals).all() else " ".join(vals.astype(str))

def detect_numerical_conflicts(df, key, numer_cols):
    """ Detects numerical conflicts when grouping on a dataframe by the given 
    keys - filling in the multiple conflict values for inspection.
    
    Args:
        df: Dataframe.
        key: Key or keys that are used for grouping.
        numer_cols: Name of numerical columns which are checked for conflicts
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values 
        delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            numer_col : numer_conflict_aggr for numer_col in numer_cols
        }
    )

def one_to_one_conflict_aggr(vals):
    """ Aggregator for the 1t1 conflicts, NaN indicates valid as 1 mapped, else 
    the conflicting values of the second identifier are returned 
    (nothing or multiple).
    
    Args:
        vals: Series that is aggregated.
    Returns:
        NaN if valid, else the conflict values joined together in a string, 
        separated by spaces.
    """
    # simple short-circuit on apparent validity
    if vals.count() == 1:
        return np.nan
    # checking if there is conflict by there being no mapping
    if vals.count() == 0:
        return ""

    # otherwise, we have multiple entries (>1)
    vals = vals.dropna() # remove null bloat to leave just the entries

    if (vals.iloc[0] == vals).all(): # reduces to 1 actual mapping, hence valid
        return np.nan
    else: # otherwise, multiple actual mappings, which creates conflicts
        return " ".join(vals.astype(str))

def detect_1t1_conflicts(df, key, oth_id):
    """ Detects one to one mapping conflicts when grouping on a dataframe by the 
    first identifier and seeing how many of the second identifier this maps to.
    
    Args:
        df: Dataframe.
        key: Key for the first identifier.
        oth_id: Name of the column for the second identifier
    
    Returns:
        Dataframe with values as NaN if no conflicts, else the conflict values, 
        for the second identifier, delimited by spaces are filled in.
    """

    return df.groupby(key).agg(
        {
            oth_id : one_to_one_conflict_aggr, 
        }
    )

**Numerical value conflicts in environmental metrics when grouping by gvkey and 
fiscal year.**

In [689]:
raw_emissions_gvkey_numer_conflicts = detect_numerical_conflicts(
    raw_emissions_df, 
    ["gvkey", "fiscalyear", "periodenddate"], 
    ["di_319413", "di_319414", "di_319415"]
)

raw_emissions_gvkey_numer_conflicts

di_319413 di_319414 di_319415
gvkey  fiscalyear periodenddate                              
001004 2022       2022-05-31          NaN       NaN       NaN
001045 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
001050 2022       2022-12-31          NaN       NaN       NaN
       2023       2023-12-31          NaN       NaN       NaN
...                                   ...       ...       ...
362683 2022       2022-03-31          NaN       NaN       NaN
362705 2022       2022-12-31          NaN       NaN       NaN
362758 2022       2022-03-31          NaN       NaN       NaN
362761 2022       2022-03-31          NaN       NaN       NaN
       2023       2023-03-31          NaN       NaN       NaN

[33896 rows x 3 columns]

In [690]:
raw_emissions_gvkey_numer_conflicts[
    pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319413"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319414"])
    | pd.notnull(raw_emissions_gvkey_numer_conflicts["di_319415"])
]

di_319413  \
gvkey  fiscalyear periodenddate                              
003413 2022       2022-12-31        2867127.0 51062759.241   
003897 2022       2022-12-31     27114550.031 25358937.262   
004094 2022       2022-12-31      810.06 1410.524 7995.579   
005180 2022       2022-12-31        636360.11 53101.342601   
005600 2022       2022-12-31      1080082.1763 4073314.057   
...                                                    ...   
275535 2022       2022-03-31          734.626 112.41037212   
       2023       2023-03-31          611.138 80.405872624   
289724 2022       2022-12-31          101.653 0.0053786782   
295786 2022       2022-12-31       134627.057 0.9207597207   
318474 2022       2022-12-31            916.0 626.35933659   

                                               di_319414  \
gvkey  fiscalyear periodenddate                            
003413 2022       2022-12-31      1018275.0 3933.0872579   
003897 2022       2022-12-31       197000.0 1953.2613298   
004094 2022       2022-12-31     7696.0 2956.39 9723.273   
005180 2022       2022-12-31       251038.0 32063.589498   
005600 2022       2022-12-31        61712.663909 73116.0   
...                                                  ...   
275535 2022       2022-03-31        4498.412 111.2256901   
       2023       2023-03-31       3819.836 79.558482923   
289724 2022       2022-12-31       1642.732 0.0869208615   
295786 2022       2022-12-31     129177.195 3.5451698548   
318474 2022       2022-12-31         4813.0 1649.4464065   

                                                      di_319415  
gvkey  fiscalyear periodenddate                                  
003413 2022       2022-12-31           1650402.398 2024652.4121  
003897 2022       2022-12-31           4025432.601 1005488.8192  
004094 2022       2022-12-31     197193.363 84877.747 39817.706  
005180 2022       2022-12-31           2436669.603 254265.62927  
005600 2022       2022-12-31            850262.94718 353150.506  
...                                                         ...  
275535 2022       2022-03-31              8751.123 2720.5732053  
       2023       2023-03-31              6836.914 1945.9953604  
289724 2022       2022-12-31              7071.309 0.3741598245  
295786 2022       2022-12-31           5009667.962 64.639735408  
318474 2022       2022-12-31            109611.983 3010.0308692  

[95 rows x 3 columns]

It is evident from the above that the gvkeys are mapping to multiple company ids in 
the same period and this is leading to multiple instances of the environment metrics

**Gvkey <-> Companyid Mapping**

- *gvkey -> company id*

In [691]:
raw_emissions_gvkey_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "companyid"
)

raw_emissions_gvkey_companyid_1t1_conflicts

,companyid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [692]:
raw_emissions_gvkey_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_companyid_1t1_conflicts["companyid"])
]

,companyid
gvkey,
002856,259777 259777 3103613 3103613
003413,263295 6020983 6020983
003897,266598 1067744 1067744
004094,528325 27755 386000 386000
005180,275442 275442 5546439
...,...
275535,109366051 109366051 881331 881331
275839,53336883 5482350
289724,215005509 215005509 30941174


Notice that gvkey can map to multiple companyids, but remember that it 
maps to at least 1 company id because it backed by companyid.

- *company id -> gvkey*

In [693]:
raw_emissions_companyid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "gvkey"
)

raw_emissions_companyid_gvkey_1t1_conflicts

,gvkey
companyid,
100000307,
100013,NaN
1000212,
1000277,
10004497,NaN
...,...
99996472,
99996476,
99996998,


In [694]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
companyid,
100000307,
1000212,
1000277,
10004521,
10005029,
...,...
99996472,
99996476,
99996998,


In [695]:
raw_emissions_companyid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_companyid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
companyid,
104422749,037090 184982
106683684,285220 316585
1067744,003897 065089 003897 065089
1073371,100557 220942 100557 220942
112732,024616 145471 024616 145471
...,...
9683016,220688 247655
983017,007824 145270 007824 145270
9833116,242985 321595


Notice that companyid can both map to no or multiple gvkeys

**Institutionid <-> Companyid Mapping**

- *companyid -> institutionid*

In [696]:
raw_emissions_companyid_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "companyid", "institutionid"
)

raw_emissions_companyid_institutionid_1t1_conflicts

,institutionid
companyid,
100000307,NaN
100013,NaN
1000212,NaN
1000277,NaN
10004497,NaN
...,...
99996472,NaN
99996476,NaN
99996998,NaN


In [697]:
raw_emissions_companyid_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_companyid_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
companyid,


Hence, we can see companyid maps to exactly one institutionid.

- *institutionid -> companyid*

In [698]:
raw_emissions_institutionid_companyid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "companyid"
)

raw_emissions_institutionid_companyid_1t1_conflicts

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11654,NaN
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
12062,NaN
...,...
118526590,NaN
118706040,NaN
118918440,NaN


In [699]:
raw_emissions_institutionid_companyid_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_companyid_1t1_conflicts["companyid"])
]

,companyid
institutionid,
11489,6520204 24951392 43964734 6520204 24951392 439...
11679,8333444 24586834 8333444 24586834
11894,7925667 34873695 7925667 34873695
13644,26465936 60478967 26465936 60478967
13959,246652 6167099 242354247 246652 6167099 242354247
...,...
109987465,
110365423,
111445983,


Here, we can see that while a company belongs to exactly one institution, an 
institution maps to many companys (none, one or potentially many).

**Gvkey <-> Institutionid Mapping**

- *gvkey -> institutionid*

In [700]:
raw_emissions_gvkey_institutionid_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "gvkey", "institutionid"
)

raw_emissions_gvkey_institutionid_1t1_conflicts

,institutionid
gvkey,
001004,NaN
001045,NaN
001050,NaN
001075,NaN
001076,NaN
...,...
362620,NaN
362683,NaN
362705,NaN


In [701]:
raw_emissions_gvkey_institutionid_1t1_conflicts[
    pd.notnull(raw_emissions_gvkey_institutionid_1t1_conflicts["institutionid"])
]

,institutionid
gvkey,
002856,4057039 4057039 4057076 4057076
003413,4057041 4057080 4057080
003897,4057044 4057083 4057083
004094,108462 4021861 4388004 4388004
005180,4806213 4806213 5053995
...,...
275535,4265945 4265945 4326804 4326804
275839,4295672 6343205
289724,4772912 4772912 6393031


Noting that there are no null values in institutionid, this means that gvkey 
corresponds to at least one, and potentially many, institutions.

- *institutionid -> gvkey*

In [702]:
raw_emissions_institutionid_gvkey_1t1_conflicts = detect_1t1_conflicts(
    raw_emissions_df, 
    "institutionid", "gvkey"
)

raw_emissions_institutionid_gvkey_1t1_conflicts

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [703]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] == "")
]

,gvkey
institutionid,
11489,
11654,
11679,
11894,
12062,
...,...
118526590,
118706040,
118918440,


In [704]:
raw_emissions_institutionid_gvkey_1t1_conflicts[
    pd.notnull(raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"]) 
    & (raw_emissions_institutionid_gvkey_1t1_conflicts["gvkey"] != "")
]

,gvkey
institutionid,
100165,002001 002002
100259,004708 027665 004708 027665
100391,008119 017095 008119 017095
101674,005849 039571
103042,012124 027867
...,...
6618361,204440 247501
6626040,278266 340246
6932029,100787 326859


We can see that institutionids map to many gvkeys (can be zero, one or multiple)

**Determine the number of gvkey identifiers that exist across all fiscal years and also the number that exist in each fiscal year**

First check whether `fiscalyear` is ever null

In [705]:
print("Is fiscalyear ever null:")
print('YES' if raw_emissions_df["fiscalyear"].isnull().any() else 'NO')

Is fiscalyear ever null:
NO


Check if there are any duplicate fiscal year entries for gvkeys

In [706]:
raw_emissions_gvkey_fiscalyear_dup = raw_emissions_df.groupby("gvkey").agg(
    {
        "fiscalyear": lambda vals: vals.duplicated().any(),
    }
)

raw_emissions_gvkey_fiscalyear_dup[
    raw_emissions_gvkey_fiscalyear_dup["fiscalyear"]
]

,fiscalyear
gvkey,
002856,True
003413,True
003897,True
004094,True
005180,True
...,...
271134,True
275535,True
289724,True


We can see there are, and this is due to the duplication from the varying 
`companyid`'s and `institutionid`'s

### Data Preparation
> Mainly involves filtering

In [870]:
cid_gvkey_df = pd.read_csv("cid_gvkey_map.csv")

cid_gvkey_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
24388,1849817514,361808,B,E,Kawan Renergy Berhad
24389,1855364409,358709,B,E,ELSA Solutions S.p.A.
24390,1855399529,358653,B,E,"SEIYU KOGYO Co.,Ltd."
24391,1859487646,359029,B,E,KET Inc.


Inspect the loaded datatypes

In [871]:
cid_gvkey_df.dtypes

companyid       int64
gvkey           int64
startdate      object
enddate        object
companyname    object
dtype: object

In [872]:
print("-- Null Values:")
cid_gvkey_df.isnull().sum()

-- Null Values:


companyid      0
gvkey          0
startdate      0
enddate        0
companyname    0
dtype: int64

Filter the mappings to just those that are one-to-one, and keep note of these.

In [873]:
cid_gvkey_1t1_df = cid_gvkey_df[(cid_gvkey_df["startdate"] == "B") & (cid_gvkey_df["enddate"] == "E")]

cid_gvkey_1t1_df

,companyid,gvkey,startdate,enddate,companyname
0,18511,210835,B,E,3i Group plc
1,18527,210418,B,E,ABB Ltd
2,18671,29751,B,E,Albemarle Corporation
3,18711,28349,B,E,The Allstate Corporation
4,18749,64768,B,E,"Amazon.com, Inc."
...,...,...,...,...,...
24388,1849817514,361808,B,E,Kawan Renergy Berhad
24389,1855364409,358709,B,E,ELSA Solutions S.p.A.
24390,1855399529,358653,B,E,"SEIYU KOGYO Co.,Ltd."
24391,1859487646,359029,B,E,KET Inc.


*Another approach for filtering mappings is considering where gvkey is not 
duplicated, as opposed to mappings that exist for all of time. This alternative 
approach may however lead to cases where the same company exists under 
different gvkeys (same companyid corresponds to multiple gvkeys). This was not 
chosen in our case.*

Verify that these mappings are indeed one-to-one.

In [874]:
print(f"Gvkey duplicates: {cid_gvkey_1t1_df['gvkey'].duplicated().any()}")
print(f"Companyid duplicates: {cid_gvkey_1t1_df['companyid'].duplicated().any()}")

Gvkey duplicates: False
Companyid duplicates: False


The lack of duplicates on both sides indicates that the mappings are indeed one to one.

>Notice filtering to one-to-one mappings reduces the number of unique gvkeys from ~24k to ~23k in the join table.

**Filter the main environment table using this join table, remembering the following facts:**
The raw main environment table has:
- ~1.8 million rows
- ~1.7 million unique companyid/institutionid values
- ~24k unique gvkey values

From the main environment table, filter entries to just those with gvkeys.

In [875]:
emissions_df = raw_emissions_df[pd.notnull(raw_emissions_df["gvkey"])]

**Further filter to just the gvkeys that 1 to 1 map with companyid.** 
This should leave one entry per fiscal year for each gvkey. 

*Note some gvkeys may not have an entry for certain fiscal years because that data is simply missing.*

In [876]:
emissions_df = emissions_df[
    np.in1d(emissions_df["gvkey"], cid_gvkey_1t1_df["gvkey"].astype(str))
]

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


Here, we check whether the above statement of exactly one entry per gvkey, 
fiscal year is true (barring the exception of missing data).

In [877]:
emissions_gvkey_fiscalyear_entries = emissions_df.groupby(["gvkey", "fiscalyear"]).size()

emissions_gvkey_fiscalyear_entries

gvkey   fiscalyear
100001  2022          1
        2023          1
100006  2022          1
100010  2022          1
100012  2022          1
                     ..
362683  2022          1
362705  2022          1
362758  2022          1
362761  2022          1
        2023          1
Length: 26407, dtype: int64

In [878]:
emissions_gvkey_fiscalyear_entries[
    emissions_gvkey_fiscalyear_entries != 1
]

Series([], dtype: int64)

The empty return for a number of entries different than 1 shows that 
we have indeed made gvkey, fiscal year unique.


In [880]:
emissions_df.nunique()["gvkey"]

19352

We can see that this process has reduced the number of unique `gvkey`'s to 
~19k now.

In [881]:
emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


**Perform QC with respect to null values**

In [882]:
emissions_df.dropna()

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States
...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan


We can see that the number of rows is not reduced, therefore there are no 
null values and the data quality has been verified.

**Write out the gvkeys for linking with fundamentals and returns data**

In [243]:
write_ids(emissions_df, "gvkey", "gvkeys.txt")

**Inspect the fiscal years and the year-month accounting ends which are associated with them**

Create an extra column to isolate the month and year from the entire `periodenddate`. 
This column will be called `periodend_ym` to denote that it just retains the 
year and month information.
- This new column will be of the `period[M]` type

In [883]:
emissions_df["periodend_ym"] = emissions_df['periodenddate'].dt.to_period('M')

print(f"New Column Type: {emissions_df['periodend_ym'].dtype}")
emissions_df

New Column Type: period[M]


,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country,periodend_ym
26,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2022-12
29,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2023-12
32,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States,2022-12
33,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States,2023-12
38,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States,2022-12
...,...,...,...,...,...,...,...,...,...,...,...
1796814,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia,2022-10
1803250,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy,2022-12
1803251,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan,2022-09
1803259,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan,2022-03


Group by fiscal year and year-month ends to see which year-month ends are 
contained within each fiscal year.

In [885]:
emissions_df.groupby(["fiscalyear", "periodend_ym"]).size()

fiscalyear  periodend_ym
2021        2022-01            61
2022        2022-01           113
            2022-02           210
            2022-03          3438
            2022-04           110
            2022-05           101
            2022-06           897
            2022-07           119
            2022-08           123
            2022-09           354
            2022-10            79
            2022-11            57
            2022-12         13506
            2023-01            22
2023        2023-01            75
            2023-02           130
            2023-03          1786
            2023-04            72
            2023-05            53
            2023-06           532
            2023-07            62
            2023-08            76
            2023-09           186
            2023-10            38
            2023-11            20
            2023-12          4186
            2024-01             1
dtype: int64

**Finalise DataFrame**
- according to joining considerations

In [886]:
emissions_df = emissions_df.reset_index(drop=True)

emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country,periodend_ym
0,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2022-12
1,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2023-12
2,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States,2022-12
3,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States,2023-12
4,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States,2022-12
...,...,...,...,...,...,...,...,...,...,...,...
26402,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia,2022-10
26403,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy,2022-12
26404,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan,2022-09
26405,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan,2022-03


In [957]:
# emissions_df.set_index(
#     ["gvkey", "fiscalyear"], drop=False
# )

institutionid  fiscalyear periodenddate     di_319413  \
gvkey  fiscalyear                                                          
122594 2022                13959        2022    2022-12-31  18853.165084   
       2023                13959        2023    2023-12-31  23250.617734   
122554 2022                14193        2022    2022-12-31  16736.521444   
       2023                14193        2023    2023-12-31  19585.900250   
263562 2022                14467        2022    2022-12-31    799.396226   
...                          ...         ...           ...           ...   
361808 2022            112329550        2022    2022-10-31    489.910950   
358709 2022            113511623        2022    2022-12-31    291.908186   
358653 2022            113511665        2022    2022-09-30     12.539324   
359029 2022            114230580        2022    2022-03-31      2.811856   
362169 2022            114764179        2022    2022-12-31   1641.238617   

                      di_319414     di_319415   companyid   gvkey  \
gvkey  fiscalyear                                                   
122594 2022        72589.700681  1.323541e+06      246652  122594   
       2023        89521.063147  1.632254e+06      246652  122594   
122554 2022        51330.155513  7.097935e+05      417155  122554   
       2023        60069.071644  8.306352e+05      417155  122554   
263562 2022         2451.712127  3.390228e+04      675974  263562   
...                         ...           ...         ...     ...   
361808 2022         1022.660743  6.379460e+03  1849817514  361808   
358709 2022          183.331947  5.197957e+02  1855364409  358709   
358653 2022            0.790235  6.400957e+00  1855399529  358653   
359029 2022            0.177205  1.435370e+00  1859487646  359029   
362169 2022         1117.543920  4.313828e+04  1863445043  362169   

                                                      companyname  \
gvkey  fiscalyear                                                   
122594 2022        State Farm Mutual Automobile Insurance Company   
       2023        State Farm Mutual Automobile Insurance Company   
122554 2022                United Services Automobile Association   
       2023                United Services Automobile Association   
263562 2022                       Everlake Life Insurance Company   
...                                                           ...   
361808 2022                                  Kawan Renergy Berhad   
358709 2022                                 ELSA Solutions S.p.A.   
358653 2022                                  SEIYU KOGYO Co.,Ltd.   
359029 2022                                              KET Inc.   
362169 2022         Chaosua Foods Industry Public Company Limited   

                         country periodend_ym  
gvkey  fiscalyear                              
122594 2022        United States      2022-12  
       2023        United States      2023-12  
122554 2022        United States      2022-12  
       2023        United States      2023-12  
263562 2022        United States      2022-12  
...                          ...          ...  
361808 2022             Malaysia      2022-10  
358709 2022                Italy      2022-12  
358653 2022                Japan      2022-09  
359029 2022                Japan      2022-03  
362169 2022             Thailand      2022-12  

[26407 rows x 11 columns]

- todo: extract fundamentals data of the same fiscal years (also see the link between periodenddate and fiscal year)
- todo: market data of one year ahead from the fiscal/fundamentals data
  - or at least one month

**Phase 1 Context Awareness**

Number of US based companies out of the ~19k unique gvkeys

In [887]:
(emissions_df.groupby("gvkey").first()["country"] == "United States").sum()

809

## Currency Data
- currency exchange fluctuations, when taking USD as the base currency, cause FX returns that should be accounted for
- market value and other quantities must be converted into a common currency (USD) to facilitate comparisons

Load daily currency exchange rates to USD & GBP. Sort by origin currency and then date.

In [1249]:
exrts_df = pd.read_csv(
    "phase1_exrts.csv", 
    parse_dates=["datadate"], 
)
exrts_df = exrts_df.sort_values(["curd", "datadate"])

exrts_df

,curd,datadate,exratd_toGBP,exratd_toUSD
0,AED,2021-01-01,0.199092,0.272239
154,AED,2021-01-02,0.199092,0.272239
308,AED,2021-01-03,0.199092,0.272239
462,AED,2021-01-04,0.200337,0.272257
616,AED,2021-01-05,0.200024,0.272253
...,...,...,...,...
202997,ZWL,2024-08-14,0.002350,0.003019
203150,ZWL,2024-08-15,0.002350,0.003023
203303,ZWL,2024-08-16,0.002350,0.003032
203456,ZWL,2024-08-17,0.002350,0.003032


In [1250]:
exrts_df.dtypes

curd                    object
datadate        datetime64[ns]
exratd_toGBP           float64
exratd_toUSD           float64
dtype: object

Compute monthly currency rates.
- Based on the exchange rates at the end of the months i.e. the last ones.

In [1251]:
exrts_df["data_ym"] = exrts_df["datadate"].dt.to_period('M')

exrts_df

,curd,datadate,exratd_toGBP,exratd_toUSD,data_ym
0,AED,2021-01-01,0.199092,0.272239,2021-01
154,AED,2021-01-02,0.199092,0.272239,2021-01
308,AED,2021-01-03,0.199092,0.272239,2021-01
462,AED,2021-01-04,0.200337,0.272257,2021-01
616,AED,2021-01-05,0.200024,0.272253,2021-01
...,...,...,...,...,...
202997,ZWL,2024-08-14,0.002350,0.003019,2024-08
203150,ZWL,2024-08-15,0.002350,0.003023,2024-08
203303,ZWL,2024-08-16,0.002350,0.003032,2024-08
203456,ZWL,2024-08-17,0.002350,0.003032,2024-08


In [1252]:
m_exrts = exrts_df.groupby(
    ["curd", "data_ym"]
).last(
).drop(
    columns="datadate"
)

m_exrts

exratd_toGBP  exratd_toUSD
curd data_ym                            
AED  2021-01      0.198491      0.272251
     2021-02      0.195503      0.272258
     2021-03      0.197625      0.272248
     2021-04      0.197083      0.272251
     2021-05      0.191637      0.272259
...                    ...           ...
ZWL  2024-04      0.002350      0.002944
     2024-05      0.002350      0.002991
     2024-06      0.002350      0.002970
     2024-07      0.002350      0.003018
     2024-08      0.002350      0.003032

[6757 rows x 2 columns]

## Market Data
- Global Market Index Data

In [976]:
mkt_df = pd.read_csv(
    "phase1_index.csv", 
    parse_dates=["datadate"], 
)
# year-month index
mkt_df["data_ym"] = mkt_df["datadate"].dt.to_period("M")
mkt_df = mkt_df.set_index("data_ym")
# compute monthly returns from prices
mkt_df["m_mktret"] = mkt_df["prccm"].pct_change()
mkt_df = mkt_df.dropna()

mkt_df

,gvkeyx,prccm,datadate,conm,tic,m_mktret
data_ym,,,,,,
2022-01,150918,3379.0814,2022-01-31,S&P Global 1200 Index,I6UNK112,-0.046984
2022-02,150918,3283.8722,2022-02-28,S&P Global 1200 Index,I6UNK112,-0.028176
2022-03,150918,3361.5970,2022-03-31,S&P Global 1200 Index,I6UNK112,0.023669
2022-04,150918,3089.8620,2022-04-30,S&P Global 1200 Index,I6UNK112,-0.080835
2022-05,150918,3097.8104,2022-05-31,S&P Global 1200 Index,I6UNK112,0.002572
2022-06,150918,2827.3566,2022-06-30,S&P Global 1200 Index,I6UNK112,-0.087305
2022-07,150918,3032.1650,2022-07-31,S&P Global 1200 Index,I6UNK112,0.072438
2022-08,150918,2899.3630,2022-08-31,S&P Global 1200 Index,I6UNK112,-0.043798
2022-09,150918,2619.9194,2022-09-30,S&P Global 1200 Index,I6UNK112,-0.096381


In [979]:
mkt_df = mkt_df[["m_mktret"]]

mkt_df

,m_mktret
data_ym,
2022-01,-0.046984
2022-02,-0.028176
2022-03,0.023669
2022-04,-0.080835
2022-05,0.002572
2022-06,-0.087305
2022-07,0.072438
2022-08,-0.043798
2022-09,-0.096381


## Returns Data

### NA & Global - Loading and Preliminary Inspection

**NA**

In [1118]:
na_returns_df = pd.read_csv(
    "phase1_returns.csv", 
    parse_dates=["datadate"], 
)

na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom
0,100001,01,2022-02-28,ACCOR SA,1.0,USD,35.01,NaN,2.618560e+08
1,100001,01,2022-03-31,ACCOR SA,NaN,USD,NaN,1.0000,2.618560e+08
2,100001,01,2022-06-30,ACCOR SA,1.0,USD,26.45,NaN,2.629960e+08
3,100001,01,2022-07-31,ACCOR SA,NaN,USD,NaN,1.0000,2.629960e+08
4,100001,01,2023-07-31,ACCOR SA,1.0,USD,37.90,NaN,2.648940e+08
...,...,...,...,...,...,...,...,...,...
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.53,1.0253,9.132301e+09
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.44,1.0253,9.132301e+09
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.26,1.0253,9.132301e+09
65259,353444,90,2024-07-31,HALEON PLC,1.0,USD,9.15,1.0253,9.132301e+09


In [893]:
na_returns_df.dtypes

gvkey                int64
iid                 object
datadate    datetime64[ns]
conm                object
ajexm              float64
curcdm              object
prccm              float64
trfm               float64
cshom              float64
dtype: object

Number of unique companies in the raw NA return data

In [894]:
na_returns_df["gvkey"].nunique()

1856

**Global**

In [895]:
global_returns_df = pd.read_csv("phase2_returns.csv")

global_returns_df

,gvkey,datadate,ajexm,prccm,dvpsxm
0,100001,2021-12-31,1.0,28.45,0.0
1,100001,2022-01-31,1.0,32.36,0.0
2,100001,2022-02-28,1.0,30.59,0.0
3,100001,2022-03-31,1.0,29.23,0.0
4,100001,2022-04-30,1.0,31.53,0.0
...,...,...,...,...,...
727700,362097,2024-04-30,1.0,NaN,0.0
727701,362097,2024-05-31,1.0,NaN,0.0
727702,362097,2024-06-30,1.0,156.95,0.0
727703,362097,2024-07-31,1.0,173.66,0.0


Number of unique companies in the raw global return data

In [896]:
global_returns_df["gvkey"].nunique()

14896

**Remarks**

*Notice that across both the NA and Global return data, we make up almost 17k 
out the ~19k companies in the emissions data*

Determine the number of companies that are in both the NA and Global datasets

In [1119]:
na_returns_uniq_gvkey = na_returns_df["gvkey"].unique()
global_returns_uniq_gvkey = global_returns_df["gvkey"].unique()

na_returns_uniq_gvkey[
    np.isin(na_returns_uniq_gvkey, global_returns_uniq_gvkey)
].size

972

### NA - Data Preparation

In [898]:
na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom
0,100001,01,2022-02-28,ACCOR SA,1.0,USD,35.01,NaN,2.618560e+08
1,100001,01,2022-03-31,ACCOR SA,NaN,USD,NaN,1.0000,2.618560e+08
2,100001,01,2022-06-30,ACCOR SA,1.0,USD,26.45,NaN,2.629960e+08
3,100001,01,2022-07-31,ACCOR SA,NaN,USD,NaN,1.0000,2.629960e+08
4,100001,01,2023-07-31,ACCOR SA,1.0,USD,37.90,NaN,2.648940e+08
...,...,...,...,...,...,...,...,...,...
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.53,1.0253,9.132301e+09
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.44,1.0253,9.132301e+09
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.26,1.0253,9.132301e+09
65259,353444,90,2024-07-31,HALEON PLC,1.0,USD,9.15,1.0253,9.132301e+09


**Add year-month**

Isolate the year-month from the data point dates, as the data points 
correspond to montly returns, and we are only interested in this 
level of granularity.

In [1120]:
na_returns_df["data_ym"] = na_returns_df["datadate"].dt.to_period('M')

na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,data_ym
0,100001,01,2022-02-28,ACCOR SA,1.0,USD,35.01,NaN,2.618560e+08,2022-02
1,100001,01,2022-03-31,ACCOR SA,NaN,USD,NaN,1.0000,2.618560e+08,2022-03
2,100001,01,2022-06-30,ACCOR SA,1.0,USD,26.45,NaN,2.629960e+08,2022-06
3,100001,01,2022-07-31,ACCOR SA,NaN,USD,NaN,1.0000,2.629960e+08,2022-07
4,100001,01,2023-07-31,ACCOR SA,1.0,USD,37.90,NaN,2.648940e+08,2023-07
...,...,...,...,...,...,...,...,...,...,...
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.53,1.0253,9.132301e+09,2024-04
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.44,1.0253,9.132301e+09,2024-05
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.26,1.0253,9.132301e+09,2024-06
65259,353444,90,2024-07-31,HALEON PLC,1.0,USD,9.15,1.0253,9.132301e+09,2024-07


Missing values must be addressed - by imputating where possible first, and then 
finally dropping rows (which have null values).
- For imputation, we must determine which variables have missing values and 
decide on the appropriate corrective action for each.

In [1121]:
print("-- Missing Values:")
na_returns_df.isnull().sum()

-- Missing Values:


gvkey          0
iid            0
datadate       0
conm           0
ajexm        610
curcdm       785
prccm       1225
trfm         942
cshom       2446
data_ym        0
dtype: int64

We can see there are missing values in the close prices, adjustment factors and total return factors.

**Imputation**

In [856]:
""" TODO: nothing right now, but can look into what imputation is possible later """

""" if prices and 
adjustment factors are enough to calculate returns, as long as the total return 
factor is all null or all non-null for each issue. """

' TODO: nothing right now, but can look into what imputation is possible later '

**Drop Rows**

In [1122]:
na_returns_df = na_returns_df.dropna()

na_returns_df

,gvkey,iid,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,data_ym
6,100001,91,2021-12-31,ACCOR SA,1.0,USD,6.471,2.1735,2.618560e+08,2021-12
7,100001,91,2022-01-31,ACCOR SA,1.0,USD,7.255,2.1735,2.618560e+08,2022-01
8,100001,91,2022-02-28,ACCOR SA,1.0,USD,6.872,2.1735,2.618560e+08,2022-02
9,100001,91,2022-03-31,ACCOR SA,1.0,USD,6.505,2.1735,2.618560e+08,2022-03
10,100001,91,2022-04-30,ACCOR SA,1.0,USD,6.653,2.1735,2.618560e+08,2022-04
...,...,...,...,...,...,...,...,...,...,...
65255,353444,90,2024-03-31,HALEON PLC,1.0,USD,8.490,1.0253,9.132301e+09,2024-03
65256,353444,90,2024-04-30,HALEON PLC,1.0,USD,8.530,1.0253,9.132301e+09,2024-04
65257,353444,90,2024-05-31,HALEON PLC,1.0,USD,8.440,1.0253,9.132301e+09,2024-05
65258,353444,90,2024-06-30,HALEON PLC,1.0,USD,8.260,1.0253,9.132301e+09,2024-06


Check if the issues have distinct data for each year-month now.

In [1123]:
na_returns_issue_ym_entries = na_returns_df.groupby(["gvkey", "iid", "data_ym"]).size()

na_returns_issue_ym_entries

gvkey   iid  data_ym
100001  91   2021-12    1
             2022-01    1
             2022-02    1
             2022-03    1
             2022-04    1
                       ..
353444  90   2024-03    1
             2024-04    1
             2024-05    1
             2024-06    1
             2024-07    1
Length: 62212, dtype: int64

In [1124]:
na_returns_issue_ym_entries[
    na_returns_issue_ym_entries != 1
]

Series([], dtype: int64)

The empty series return indicates that this is indeed the case.

**Reindex & Sort**
- by `gvkey`, `iid` and then `data_ym` 
- we know `data_ym` is particularly appropriate 
for indexing now due to its uniqueness within security issues

In [1125]:
na_returns_df = na_returns_df.set_index(["gvkey", "iid", "data_ym"])
na_returns_df = na_returns_df.sort_index()

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom  
gvkey  iid data_ym                
100001 91  2021-12  2.618560e+08  
           2022-01  2.618560e+08  
           2022-02  2.618560e+08  
           2022-03  2.618560e+08  
           2022-04  2.618560e+08  
...                          ...  
353444 90  2024-03  9.132301e+09  
           2024-04  9.132301e+09  
           2024-05  9.132301e+09  
           2024-06  9.132301e+09  
           2024-07  9.132301e+09  

[62212 rows x 7 columns]

**Calculate returns.**
- Utilising the year-month entries of each security issue

Check the number of year-month data records for each issue

In [1127]:
na_returns_issue_yms = na_returns_df.reset_index(
).groupby(
    ["gvkey", "iid"]
).agg(
    {
        "data_ym": lambda vals: vals.count()
    }
)

na_returns_issue_yms

data_ym
gvkey  iid         
100001 91        32
100012 01        32
100013 90        32
100022 01         2
       02        16
...             ...
350952 90        31
351491 01        31
351590 01        18
       90        32
353444 90        25

[2172 rows x 1 columns]

We can see there are 2204 issues from <1900 companies with varying numbers of 
data points.

In [1128]:
na_returns_df.xs((100022, "01"), level=("gvkey", "iid"))

,datadate,conm,ajexm,curcdm,prccm,trfm,cshom
data_ym,,,,,,,
2022-07,2022-07-31,BAYER MOTOREN WERKE AG,1.0,USD,73.8,1.0,59404000.0
2024-02,2024-02-29,BAYER MOTOREN WERKE AG,1.0,USD,109.5,1.0,60844000.0


We also observe that the year-month entries of security issues can be 
non-contiguous when they are incomplete.

In [1129]:
na_returns_max_issue_yms = na_returns_issue_yms["data_ym"].max()
print(f"Maximum issue data points: {na_returns_max_issue_yms}")

na_returns_full_issue = na_returns_issue_yms[
    na_returns_issue_yms["data_ym"] == na_returns_max_issue_yms
]

na_returns_full_issue

Maximum issue data points: 32


,,data_ym
gvkey,iid,
100001,91,32
100012,01,32
100013,90,32
100022,90,32
100045,90,32
...,...,...
347328,90,32
347708,90,32
349485,90,32


We observe there are fewer issues with the maximum number of data points.

In [1130]:
na_returns_df = na_returns_df.copy()
na_returns_df = na_returns_df.sort_index()

Calculate returns as per the Compustat manual. To do this we first need 
adjusted close.

In [1131]:
na_returns_df["adjclose"] = (na_returns_df["prccm"] / na_returns_df["ajexm"]) * na_returns_df["trfm"]

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  
gvkey  iid data_ym                           
100001 91  2021-12  2.618560e+08  14.064719  
           2022-01  2.618560e+08  15.768743  
           2022-02  2.618560e+08  14.936292  
           2022-03  2.618560e+08  14.138618  
           2022-04  2.618560e+08  14.460296  
...                          ...        ...  
353444 90  2024-03  9.132301e+09   8.704797  
           2024-04  9.132301e+09   8.745809  
           2024-05  9.132301e+09   8.653532  
           2024-06  9.132301e+09   8.468978  
           2024-07  9.132301e+09   9.381495  

[62212 rows x 8 columns]

Exchange rates to the home currency (USD) are joined to allow computation of FX 
returns - that form a part of the overall return.

In [1132]:
na_returns_df = na_returns_df.reset_index().merge(
    m_exrts, 
    how="left", 
    left_on=["curcdm", "data_ym"], 
    right_index=True, 
).set_index(
    ["gvkey", "iid", "data_ym"]
)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  
gvkey  iid data_ym                                                       
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0  
           2022-01  2.618560e+08  15.768743      0.745101           1.0  
           2022-02  2.618560e+08  14.936292      0.745490           1.0  
           2022-03  2.618560e+08  14.138618      0.760746           1.0  
           2022-04  2.618560e+08  14.460296      0.796622           1.0  
...                          ...        ...           ...           ...  
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  
           2024-04  9.132301e+09   8.745809      0.798403           1.0  
           2024-05  9.132301e+09   8.653532      0.785916           1.0  
           2024-06  9.132301e+09   8.468978      0.791264           1.0  
           2024-07  9.132301e+09   9.381495      0.778756           1.0  

[62212 rows x 10 columns]

From the adjusted closes and exchange rates, we can now compute the local and FX returns.

In [1133]:
na_returns_df = pd.concat(
    [
        na_returns_df, 
        na_returns_df.groupby(
            level=["gvkey", "iid"]
        )[["adjclose", "exratd_toUSD", "exratd_toGBP"]].pct_change(
        ).rename(
            columns={
                "adjclose": "local_ret", 
                "exratd_toUSD": "USD_fxret", 
                "exratd_toGBP": "GBP_fxret", 
            }
        )
    ], 
    axis=1
)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret  
gvkey  iid data_ym                                   
100001 91  2021-12        NaN        NaN        NaN  
           2022-01   0.121156        0.0   0.008941  
           2022-02  -0.052791        0.0   0.000522  
           2022-03  -0.053405        0.0   0.020464  
           2022-04   0.022752        0.0   0.047160  
...                       ...        ...        ...  
353444 90  2024-03   0.004163        0.0   0.000238  
           2024-04   0.004711        0.0   0.008383  
           2024-05  -0.010551        0.0  -0.015640  
           2024-06  -0.021327        0.0   0.006805  
           2024-07   0.107748        0.0  -0.015809  

[62212 rows x 13 columns]

Combine local and FX returns to obtain USD returns. Note that these are 
raw returns with varying frequencies.

In [1134]:
na_returns_df["USD_ret"] = (
    (1 + na_returns_df["local_ret"]) * (1 + na_returns_df["USD_fxret"]) - 1
)
na_returns_df["GBP_ret"] = (
    (1 + na_returns_df["local_ret"]) * (1 + na_returns_df["GBP_fxret"]) - 1
)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  
gvkey  iid data_ym                                                       
100001 91  2021-12        NaN        NaN        NaN       NaN       NaN  
           2022-01   0.121156        0.0   0.008941  0.121156  0.131180  
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297  
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034  
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985  
...                       ...        ...        ...       ...       ...  
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401  
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134  
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026  
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667  
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236  

[62212 rows x 15 columns]

The differences in months of these raw returns can be used to standardise them 
to a common frequency (monthly).

In [1135]:
""" Differences in months, that can be used to standardise returns to the 
same frequency - monthly """
na_returns_df["ret_mfreq"] = na_returns_df.reset_index(
).groupby(
    ["gvkey", "iid"]
)["data_ym"].diff(
).apply(
    lambda x: x.n if pd.notnull(x) else np.nan
).values

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  \
gvkey  iid data_ym                                                        
100001 91  2021-12        NaN        NaN        NaN       NaN       NaN   
           2022-01   0.121156        0.0   0.008941  0.121156  0.131180   
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297   
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034   
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985   
...                       ...        ...        ...       ...       ...   
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401   
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134   
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026   
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667   
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236   

                    ret_mfreq  
gvkey  iid data_ym             
100001 91  2021-12        NaN  
           2022-01        1.0  
           2022-02        1.0  
           2022-03        1.0  
           2022-04        1.0  
...                       ...  
353444 90  2024-03        1.0  
           2024-04        1.0  
           2024-05        1.0  
           2024-06        1.0  
           2024-07        1.0  

[62212 rows x 16 columns]

*Sanity check - there should be 0 return month frequencies that 
are non-null for the first year-month entries of issues*

In [1136]:
pd.notnull(na_returns_df.groupby(
    level=["gvkey", "iid"]
).agg(
    {
        "ret_mfreq": lambda vals: vals.iloc[0]
    }
)).sum()

ret_mfreq    0
dtype: int64

Standardise the raw USD returns to the monthly frequency.

In [1137]:
na_returns_df["m_USD_ret"] = (
    (1 + na_returns_df["USD_ret"]) ** (1/na_returns_df["ret_mfreq"])
) - 1

na_returns_df["m_GBP_ret"] = (
    (1 + na_returns_df["GBP_ret"]) ** (1/na_returns_df["ret_mfreq"])
) - 1

na_returns_df["m_local_ret"] = (
    (1 + na_returns_df["local_ret"]) ** (1/na_returns_df["ret_mfreq"])
) - 1

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  \
gvkey  iid data_ym                                                        
100001 91  2021-12        NaN        NaN        NaN       NaN       NaN   
           2022-01   0.121156        0.0   0.008941  0.121156  0.131180   
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297   
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034   
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985   
...                       ...        ...        ...       ...       ...   
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401   
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134   
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026   
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667   
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236   

                    ret_mfreq  m_USD_ret  m_GBP_ret  m_local_ret  
gvkey  iid data_ym                                                
100001 91  2021-12        NaN        NaN        NaN          NaN  
           2022-01        1.0   0.121156   0.131180     0.121156  
           2022-02        1.0  -0.052791  -0.052297    -0.052791  
           2022-03        1.0  -0.053405  -0.034034    -0.053405  
           2022-04        1.0   0.022752   0.070985     0.022752  
...                       ...        ...        ...          ...  
353444 90  2024-03        1.0   0.004163   0.004401     0.004163  
           2024-04        1.0   0.004711   0.013134     0.004711  
           2024-05        1.0  -0.010551  -0.026026    -0.010551  
           2024-06        1.0  -0.021327  -0.014667    -0.021327  
           2024-07        1.0   0.107748   0.090236     0.107748  

[62212 rows x 19 columns]

*Notice that there are companies that have gone bankrupt (0 raw close price & 
0 adjusted close price), which leads to further null returns beyond just the 
first month for some companies (due to division by 0).*

In [1138]:
na_returns_df[na_returns_df["prccm"] == 0]

,,,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,adjclose,exratd_toGBP,exratd_toUSD,local_ret,USD_fxret,GBP_fxret,USD_ret,GBP_ret,ret_mfreq,m_USD_ret,m_GBP_ret,m_local_ret
gvkey,iid,data_ym,,,,,,,,,,,,,,,,,,,
175533,01,2024-07,2024-07-31,INFINITY PHARMACEUTICALS INC,1.0,USD,0.0,1.0,90761000.0,0.0,0.778756,1.0,-1.0,0.0,-0.009812,-1.0,-1.0,6.0,-1.0,-1.0,-1.0


In [1139]:
na_returns_df.xs((175533, "01"))

,datadate,conm,ajexm,curcdm,prccm,trfm,cshom,adjclose,exratd_toGBP,exratd_toUSD,local_ret,USD_fxret,GBP_fxret,USD_ret,GBP_ret,ret_mfreq,m_USD_ret,m_GBP_ret,m_local_ret
data_ym,,,,,,,,,,,,,,,,,,,
2021-12,2021-12-31,INFINITY PHARMACEUTICALS INC,1.0,USD,2.2500,1.0,89015000.0,2.2500,0.738498,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01,2022-01-31,INFINITY PHARMACEUTICALS INC,1.0,USD,1.2400,1.0,89015000.0,1.2400,0.745101,1.0,-0.448889,0.0,0.008941,-0.448889,-0.443961,1.0,-0.448889,-0.443961,-0.448889
2022-02,2022-02-28,INFINITY PHARMACEUTICALS INC,1.0,USD,1.1000,1.0,89015000.0,1.1000,0.745490,1.0,-0.112903,0.0,0.000522,-0.112903,-0.112440,1.0,-0.112903,-0.112440,-0.112903
2022-03,2022-03-31,INFINITY PHARMACEUTICALS INC,1.0,USD,1.1400,1.0,89015000.0,1.1400,0.760746,1.0,0.036364,0.0,0.020464,0.036364,0.057572,1.0,0.036364,0.057572,0.036364
2022-04,2022-04-30,INFINITY PHARMACEUTICALS INC,1.0,USD,0.7892,1.0,89155000.0,0.7892,0.796622,1.0,-0.307719,0.0,0.047160,-0.307719,-0.275071,1.0,-0.307719,-0.275071,-0.307719
2022-05,2022-05-31,INFINITY PHARMACEUTICALS INC,1.0,USD,0.6724,1.0,89155000.0,0.6724,0.793714,1.0,-0.147998,0.0,-0.003651,-0.147998,-0.151109,1.0,-0.147998,-0.151109,-0.147998
2022-06,2022-06-30,INFINITY PHARMACEUTICALS INC,1.0,USD,0.6323,1.0,89155000.0,0.6323,0.822978,1.0,-0.059637,0.0,0.036869,-0.059637,-0.024967,1.0,-0.059637,-0.024967,-0.059637
2022-07,2022-07-31,INFINITY PHARMACEUTICALS INC,1.0,USD,0.6357,1.0,89155000.0,0.6357,0.821895,1.0,0.005377,0.0,-0.001315,0.005377,0.004055,1.0,0.005377,0.004055,0.005377
2022-08,2022-08-31,INFINITY PHARMACEUTICALS INC,1.0,USD,1.6100,1.0,89278000.0,1.6100,0.859107,1.0,1.532641,0.0,0.045275,1.532641,1.647306,1.0,1.532641,1.647306,1.532641


In [1140]:
na_returns_df = na_returns_df.copy()
na_returns_df = na_returns_df.sort_index()

At this point, the computation of monthly returns has been completed.
- w.r.t base currencies of USD and GBP.

**Calculate monthly market capitalisation**
- standardise the currency (USD)

In [1141]:
na_returns_df["local_mktval"] = na_returns_df["cshom"] * na_returns_df["prccm"]

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  \
gvkey  iid data_ym                                                        
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0   
           2022-01  2.618560e+08  15.768743      0.745101           1.0   
           2022-02  2.618560e+08  14.936292      0.745490           1.0   
           2022-03  2.618560e+08  14.138618      0.760746           1.0   
           2022-04  2.618560e+08  14.460296      0.796622           1.0   
...                          ...        ...           ...           ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0   
           2024-04  9.132301e+09   8.745809      0.798403           1.0   
           2024-05  9.132301e+09   8.653532      0.785916           1.0   
           2024-06  9.132301e+09   8.468978      0.791264           1.0   
           2024-07  9.132301e+09   9.381495      0.778756           1.0   

                    local_ret  USD_fxret  GBP_fxret   USD_ret   GBP_ret  \
gvkey  iid data_ym                                                        
100001 91  2021-12        NaN        NaN        NaN       NaN       NaN   
           2022-01   0.121156        0.0   0.008941  0.121156  0.131180   
           2022-02  -0.052791        0.0   0.000522 -0.052791 -0.052297   
           2022-03  -0.053405        0.0   0.020464 -0.053405 -0.034034   
           2022-04   0.022752        0.0   0.047160  0.022752  0.070985   
...                       ...        ...        ...       ...       ...   
353444 90  2024-03   0.004163        0.0   0.000238  0.004163  0.004401   
           2024-04   0.004711        0.0   0.008383  0.004711  0.013134   
           2024-05  -0.010551        0.0  -0.015640 -0.010551 -0.026026   
           2024-06  -0.021327        0.0   0.006805 -0.021327 -0.014667   
           2024-07   0.107748        0.0  -0.015809  0.107748  0.090236   

                    ret_mfreq  m_USD_ret  m_GBP_ret  m_local_ret  local_mktval  
gvkey  iid data_ym                                                              
100001 91  2021-12        NaN        NaN        NaN          NaN  1.694470e+09  
           2022-01        1.0   0.121156   0.131180     0.121156  1.899765e+09  
           2022-02        1.0  -0.052791  -0.052297    -0.052791  1.799474e+09  
           2022-03        1.0  -0.053405  -0.034034    -0.053405  1.703373e+09  
           2022-04        1.0   0.022752   0.070985     0.022752  1.742128e+09  
...                       ...        ...        ...          ...           ...  
353444 90  2024-03        1.0   0.004163   0.004401     0.004163  7.753324e+10  
           2024-04        1.0   0.004711   0.013134     0.004711  7.789853e+10  
           2024-05        1.0  -0.010551  -0.026026    -0.010551  7.707662e+10  
           2024-06        1.0  -0.021327  -0.014667    -0.021327  7.543281e+10  
           2024-07        1.0   0.107748   0.090236     0.107748  8.356055e+10  

[62212 rows x 20 columns]

In [1142]:
na_returns_df["USD_mktval"] = na_returns_df["local_mktval"] * na_returns_df["exratd_toUSD"]
na_returns_df["GBP_mktval"] = na_returns_df["local_mktval"] * na_returns_df["exratd_toGBP"]

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0  ...   
           2022-01  2.618560e+08  15.768743      0.745101           1.0  ...   
           2022-02  2.618560e+08  14.936292      0.745490           1.0  ...   
           2022-03  2.618560e+08  14.138618      0.760746           1.0  ...   
           2022-04  2.618560e+08  14.460296      0.796622           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                    GBP_fxret   USD_ret   GBP_ret  ret_mfreq  m_USD_ret  \
gvkey  iid data_ym                                                        
100001 91  2021-12        NaN       NaN       NaN        NaN        NaN   
           2022-01   0.008941  0.121156  0.131180        1.0   0.121156   
           2022-02   0.000522 -0.052791 -0.052297        1.0  -0.052791   
           2022-03   0.020464 -0.053405 -0.034034        1.0  -0.053405   
           2022-04   0.047160  0.022752  0.070985        1.0   0.022752   
...                       ...       ...       ...        ...        ...   
353444 90  2024-03   0.000238  0.004163  0.004401        1.0   0.004163   
           2024-04   0.008383  0.004711  0.013134        1.0   0.004711   
           2024-05  -0.015640 -0.010551 -0.026026        1.0  -0.010551   
           2024-06   0.006805 -0.021327 -0.014667        1.0  -0.021327   
           2024-07  -0.015809  0.107748  0.090236        1.0   0.107748   

                    m_GBP_ret  m_local_ret  local_mktval    USD_mktval  \
gvkey  iid data_ym                                                       
100001 91  2021-12        NaN          NaN  1.694470e+09  1.694470e+09   
           2022-01   0.131180     0.121156  1.899765e+09  1.899765e+09   
           2022-02  -0.052297    -0.052791  1.799474e+09  1.799474e+09   
           2022-03  -0.034034    -0.053405  1.703373e+09  1.703373e+09   
           2022-04   0.070985     0.022752  1.742128e+09  1.742128e+09   
...                       ...          ...           ...           ...   
353444 90  2024-03   0.004401     0.004163  7.753324e+10  7.753324e+10   
           2024-04   0.013134     0.004711  7.789853e+10  7.789853e+10   
           2024-05  -0.026026    -0.010551  7.707662e+10  7.707662e+10   
           2024-06  -0.014667    -0.021327  7.543281e+10  7.543281e+10   
           2024-07   0.090236     0.107748  8.356055e+10  8.356055e+10   

                      GBP_mktval  
gvkey  iid data_ym                
100001 9

Shift the calculated market values one forward, in each security, to prevent look-ahead bias in return predictions.

In [1149]:
na_returns_df[
    ["local_mktval", "USD_mktval", "GBP_mktval"]
] = na_returns_df.groupby(
    level=["gvkey", "iid"]
)[
    ["local_mktval", "USD_mktval", "GBP_mktval"]
].shift()

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0  ...   
           2022-01  2.618560e+08  15.768743      0.745101           1.0  ...   
           2022-02  2.618560e+08  14.936292      0.745490           1.0  ...   
           2022-03  2.618560e+08  14.138618      0.760746           1.0  ...   
           2022-04  2.618560e+08  14.460296      0.796622           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                    GBP_fxret   USD_ret   GBP_ret  ret_mfreq  m_USD_ret  \
gvkey  iid data_ym                                                        
100001 91  2021-12        NaN       NaN       NaN        NaN        NaN   
           2022-01   0.008941  0.121156  0.131180        1.0   0.121156   
           2022-02   0.000522 -0.052791 -0.052297        1.0  -0.052791   
           2022-03   0.020464 -0.053405 -0.034034        1.0  -0.053405   
           2022-04   0.047160  0.022752  0.070985        1.0   0.022752   
...                       ...       ...       ...        ...        ...   
353444 90  2024-03   0.000238  0.004163  0.004401        1.0   0.004163   
           2024-04   0.008383  0.004711  0.013134        1.0   0.004711   
           2024-05  -0.015640 -0.010551 -0.026026        1.0  -0.010551   
           2024-06   0.006805 -0.021327 -0.014667        1.0  -0.021327   
           2024-07  -0.015809  0.107748  0.090236        1.0   0.107748   

                    m_GBP_ret  m_local_ret  local_mktval    USD_mktval  \
gvkey  iid data_ym                                                       
100001 91  2021-12        NaN          NaN           NaN           NaN   
           2022-01   0.131180     0.121156  1.694470e+09  1.694470e+09   
           2022-02  -0.052297    -0.052791  1.899765e+09  1.899765e+09   
           2022-03  -0.034034    -0.053405  1.799474e+09  1.799474e+09   
           2022-04   0.070985     0.022752  1.703373e+09  1.703373e+09   
...                       ...          ...           ...           ...   
353444 90  2024-03   0.004401     0.004163  7.904795e+10  7.904795e+10   
           2024-04   0.013134     0.004711  7.753324e+10  7.753324e+10   
           2024-05  -0.026026    -0.010551  7.789853e+10  7.789853e+10   
           2024-06  -0.014667    -0.021327  7.707662e+10  7.707662e+10   
           2024-07   0.090236     0.107748  7.543281e+10  7.543281e+10   

                      GBP_mktval  
gvkey  iid data_ym                
100001 9

**Finalise**

Augment with year columns for linking with fundamentals and emissions data.

In [1150]:
na_returns_df["datayear"] = na_returns_df.reset_index()["data_ym"].dt.year.values
na_returns_df["datayear-1"] = na_returns_df["datayear"] - 1

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0  ...   
           2022-01  2.618560e+08  15.768743      0.745101           1.0  ...   
           2022-02  2.618560e+08  14.936292      0.745490           1.0  ...   
           2022-03  2.618560e+08  14.138618      0.760746           1.0  ...   
           2022-04  2.618560e+08  14.460296      0.796622           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                     GBP_ret  ret_mfreq  m_USD_ret  m_GBP_ret  m_local_ret  \
gvkey  iid data_ym                                                           
100001 91  2021-12       NaN        NaN        NaN        NaN          NaN   
           2022-01  0.131180        1.0   0.121156   0.131180     0.121156   
           2022-02 -0.052297        1.0  -0.052791  -0.052297    -0.052791   
           2022-03 -0.034034        1.0  -0.053405  -0.034034    -0.053405   
           2022-04  0.070985        1.0   0.022752   0.070985     0.022752   
...                      ...        ...        ...        ...          ...   
353444 90  2024-03  0.004401        1.0   0.004163   0.004401     0.004163   
           2024-04  0.013134        1.0   0.004711   0.013134     0.004711   
           2024-05 -0.026026        1.0  -0.010551  -0.026026    -0.010551   
           2024-06 -0.014667        1.0  -0.021327  -0.014667    -0.021327   
           2024-07  0.090236        1.0   0.107748   0.090236     0.107748   

                    local_mktval    USD_mktval    GBP_mktval  datayear  \
gvkey  iid data_ym                                                       
100001 91  2021-12           NaN           NaN           NaN      2021   
           2022-01  1.694470e+09  1.694470e+09  1.251363e+09      2022   
           2022-02  1.899765e+09  1.899765e+09  1.415517e+09      2022   
           2022-03  1.799474e+09  1.799474e+09  1.341490e+09      2022   
           2022-04  1.703373e+09  1.703373e+09  1.295834e+09      2022   
...                          ...           ...           ...       ...   
353444 90  2024-03  7.904795e+10  7.904795e+10  6.257259e+10      2024   
           2024-04  7.753324e+10  7.753324e+10  6.138815e+10      2024   
           2024-05  7.789853e+10  7.789853e+10  6.219443e+10      2024   
           2024-06  7.707662e+10  7.707662e+10  6.057578e+10      2024   
           2024-07  7.543281e+10  7.543281e+10  5.968730e+10      2024   

                    datayear-1  
gvkey 

Join market return in.

In [1151]:
na_returns_df = na_returns_df.join(
    mkt_df, 
    how="left", 
    on="data_ym", 
)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0  ...   
           2022-01  2.618560e+08  15.768743      0.745101           1.0  ...   
           2022-02  2.618560e+08  14.936292      0.745490           1.0  ...   
           2022-03  2.618560e+08  14.138618      0.760746           1.0  ...   
           2022-04  2.618560e+08  14.460296      0.796622           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                    ret_mfreq  m_USD_ret  m_GBP_ret  m_local_ret  \
gvkey  iid data_ym                                                 
100001 91  2021-12        NaN        NaN        NaN          NaN   
           2022-01        1.0   0.121156   0.131180     0.121156   
           2022-02        1.0  -0.052791  -0.052297    -0.052791   
           2022-03        1.0  -0.053405  -0.034034    -0.053405   
           2022-04        1.0   0.022752   0.070985     0.022752   
...                       ...        ...        ...          ...   
353444 90  2024-03        1.0   0.004163   0.004401     0.004163   
           2024-04        1.0   0.004711   0.013134     0.004711   
           2024-05        1.0  -0.010551  -0.026026    -0.010551   
           2024-06        1.0  -0.021327  -0.014667    -0.021327   
           2024-07        1.0   0.107748   0.090236     0.107748   

                    local_mktval    USD_mktval    GBP_mktval  datayear  \
gvkey  iid data_ym                                                       
100001 91  2021-12           NaN           NaN           NaN      2021   
           2022-01  1.694470e+09  1.694470e+09  1.251363e+09      2022   
           2022-02  1.899765e+09  1.899765e+09  1.415517e+09      2022   
           2022-03  1.799474e+09  1.799474e+09  1.341490e+09      2022   
           2022-04  1.703373e+09  1.703373e+09  1.295834e+09      2022   
...                          ...           ...           ...       ...   
353444 90  2024-03  7.904795e+10  7.904795e+10  6.257259e+10      2024   
           2024-04  7.753324e+10  7.753324e+10  6.138815e+10      2024   
           2024-05  7.789853e+10  7.789853e+10  6.219443e+10      2024   
           2024-06  7.707662e+10  7.707662e+10  6.057578e+10      2024   
           2024-07  7.543281e+10  7.543281e+10  5.968730e+10      2024   

                    datayear-1  m_mktret  
gvkey  iid data_ym                        
100001 91  2021-12        2020       NaN  
           2022-01        2021 -0.046984

Compute rolling beta within issues.
- From rolling covariances and variances.

In [1180]:
na_returns_beta = na_returns_df.reset_index(
).groupby(
    ["gvkey", "iid"] # within issues
)[["m_USD_ret", "m_mktret"]].rolling(
    12
).cov().unstack()["m_mktret"].rename(
    columns={
        "m_USD_ret": "cov(i,m)", 
        "m_mktret": "var(m)", 
    }
).set_index(na_returns_df.index)

na_returns_beta

cov(i,m)    var(m)
gvkey  iid data_ym                    
100001 91  2021-12       NaN       NaN
           2022-01       NaN       NaN
           2022-02       NaN       NaN
           2022-03       NaN       NaN
           2022-04       NaN       NaN
...                      ...       ...
353444 90  2024-03  0.001123  0.001587
           2024-04  0.001117  0.001833
           2024-05  0.000845  0.001833
           2024-06  0.000689  0.001679
           2024-07  0.000630  0.001653

[62212 rows x 2 columns]

Once again shift the calculated betas one forward, in each security, to prevent look-ahead bias.

In [1181]:
na_returns_beta["beta"] = na_returns_beta["cov(i,m)"] / na_returns_beta["var(m)"]
na_returns_beta = na_returns_beta.drop(columns=["cov(i,m)", "var(m)"])

na_returns_beta = na_returns_beta.groupby(
    level=["gvkey", "iid"]
).shift()

na_returns_beta

beta
gvkey  iid data_ym          
100001 91  2021-12       NaN
           2022-01       NaN
           2022-02       NaN
           2022-03       NaN
           2022-04       NaN
...                      ...
353444 90  2024-03  0.736227
           2024-04  0.707596
           2024-05  0.609430
           2024-06  0.461021
           2024-07  0.410261

[62212 rows x 1 columns]

Beta is concatenated together with the returns, aligning with the index.

In [1183]:
na_returns_df = pd.concat([na_returns_df, na_returns_beta], axis=1)

na_returns_df

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2021-12 2021-12-31    ACCOR SA    1.0    USD  6.471  2.1735   
           2022-01 2022-01-31    ACCOR SA    1.0    USD  7.255  2.1735   
           2022-02 2022-02-28    ACCOR SA    1.0    USD  6.872  2.1735   
           2022-03 2022-03-31    ACCOR SA    1.0    USD  6.505  2.1735   
           2022-04 2022-04-30    ACCOR SA    1.0    USD  6.653  2.1735   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2021-12  2.618560e+08  14.064719      0.738498           1.0  ...   
           2022-01  2.618560e+08  15.768743      0.745101           1.0  ...   
           2022-02  2.618560e+08  14.936292      0.745490           1.0  ...   
           2022-03  2.618560e+08  14.138618      0.760746           1.0  ...   
           2022-04  2.618560e+08  14.460296      0.796622           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                    m_USD_ret  m_GBP_ret  m_local_ret  local_mktval  \
gvkey  iid data_ym                                                    
100001 91  2021-12        NaN        NaN          NaN           NaN   
           2022-01   0.121156   0.131180     0.121156  1.694470e+09   
           2022-02  -0.052791  -0.052297    -0.052791  1.899765e+09   
           2022-03  -0.053405  -0.034034    -0.053405  1.799474e+09   
           2022-04   0.022752   0.070985     0.022752  1.703373e+09   
...                       ...        ...          ...           ...   
353444 90  2024-03   0.004163   0.004401     0.004163  7.904795e+10   
           2024-04   0.004711   0.013134     0.004711  7.753324e+10   
           2024-05  -0.010551  -0.026026    -0.010551  7.789853e+10   
           2024-06  -0.021327  -0.014667    -0.021327  7.707662e+10   
           2024-07   0.107748   0.090236     0.107748  7.543281e+10   

                      USD_mktval    GBP_mktval  datayear  datayear-1  \
gvkey  iid data_ym                                                     
100001 91  2021-12           NaN           NaN      2021        2020   
           2022-01  1.694470e+09  1.251363e+09      2022        2021   
           2022-02  1.899765e+09  1.415517e+09      2022        2021   
           2022-03  1.799474e+09  1.341490e+09      2022        2021   
           2022-04  1.703373e+09  1.295834e+09      2022        2021   
...                          ...           ...       ...         ...   
353444 90  2024-03  7.904795e+10  6.257259e+10      2024        2023   
           2024-04  7.753324e+10  6.138815e+10      2024        2023   
           2024-05  7.789853e+10  6.219443e+10      2024        2023   
           2024-06  7.707662e+10  6.057578e+10      2024        2023   
           2024-07  7.543281e+10  5.968730e+10      2024        2023   

                    m_mktret      beta  
gvkey  iid data_ym                      
100001 91  2021-12       NaN       NaN  
           2022-01 -0.046984     

For this whole dataframe, the following should be noted:
- rolling beta computation causes many null values at the start of security entries
  - shifting to prevent look-ahead pushes in a further null
- the first security entry has null returns
- shifting market value to prevent look-ahead once again pushes in a null

In [1197]:
na_returns_df.describe()

,ajexm,prccm,trfm,cshom,adjclose,exratd_toGBP,exratd_toUSD,local_ret,USD_fxret,GBP_fxret,...,m_USD_ret,m_GBP_ret,m_local_ret,local_mktval,USD_mktval,GBP_mktval,datayear,datayear-1,m_mktret,beta
count,62212.000000,62212.000000,62212.000000,6.221200e+04,62212.000000,62212.000000,62212.000000,60040.000000,60040.000000,60040.000000,...,60040.000000,60040.000000,60040.000000,6.004000e+04,6.004000e+04,6.004000e+04,62212.000000,62212.000000,60210.000000,35501.000000
mean,1.046512,52.558090,1.667681,1.438783e+09,75.175457,0.785222,0.978986,0.105392,-0.000231,0.001887,...,0.100216,0.105342,0.100404,3.278615e+10,3.269072e+10,2.616101e+10,2022.763197,2021.763197,0.004289,-1.038176
std,1.207169,305.446796,2.599476,9.150298e+09,847.647506,0.064682,0.069400,22.391567,0.004997,0.023308,...,22.370773,23.365244,22.370770,2.334696e+11,2.334735e+11,1.861454e+11,0.816550,0.816550,0.050343,184.604302
min,0.000025,0.000000,1.000000,0.000000e+00,0.000000,0.563793,0.719976,-1.000000,-0.047641,-0.133650,...,-1.000000,-1.000000,-1.000000,0.000000e+00,0.000000e+00,0.000000e+00,2021.000000,2020.000000,-0.096381,-18390.862930
25%,1.000000,6.020000,1.000000,5.382400e+07,8.652851,0.778756,1.000000,-0.071530,0.000000,-0.015809,...,-0.071827,-0.064013,-0.071097,6.068524e+08,5.801618e+08,4.648638e+08,2022.000000,2021.000000,-0.036541,0.599524
50%,1.000000,14.947000,1.222300,1.673790e+08,22.902597,0.791766,1.000000,-0.002290,0.000000,0.000522,...,-0.002565,-0.001281,-0.002269,2.753375e+09,2.699197e+09,2.162954e+09,2023.000000,2022.000000,0.015082,1.043443
75%,1.000000,37.233500,1.659300,5.590000e+08,56.719116,0.821895,1.000000,0.068645,0.000000,0.014935,...,0.068536,0.063056,0.068360,1.129177e+10,1.122201e+10,8.984772e+09,2023.000000,2022.000000,0.042842,1.529993
max,50.000000,13583.800000,97.982600,4.392889e+11,140400.000000,0.897424,1.000000,5473.000000,0.027766,0.179664,...,5473.000000,5717.151306,5473.000000,9.830370e+12,9.830370e+12,7.259708e+12,2024.000000,2023.000000,0.089325,75.144283


We also observe extreme outliers for return, market value & beta.

In [1199]:
# na_returns_df.xs((140044, "01"))
# na_returns_df.xs((311798, "01"))

Thus, we create an instance will the null values and outlier rows filtered out.
- at the 0.1% level

In [1204]:
na_returns = na_returns_df.dropna()

na_returns = na_returns[
    (na_returns["m_USD_ret"].quantile(0.001) < na_returns["m_USD_ret"])
    & (na_returns["m_USD_ret"] < na_returns["m_USD_ret"].quantile(0.999))
    & (na_returns["beta"].quantile(0.001) < na_returns["beta"])
    & (na_returns["beta"] < na_returns["beta"].quantile(0.999))
    & (na_returns["USD_mktval"].quantile(0.001) < na_returns["USD_mktval"])
    & (na_returns["USD_mktval"] < na_returns["USD_mktval"].quantile(0.999))
]

na_returns

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2023-01 2023-01-31    ACCOR SA    1.0    USD  6.466  2.1735   
           2023-02 2023-02-28    ACCOR SA    1.0    USD  6.677  2.1735   
           2023-03 2023-03-31    ACCOR SA    1.0    USD  6.512  2.1735   
           2023-04 2023-04-30    ACCOR SA    1.0    USD  7.092  2.1735   
           2023-05 2023-05-31    ACCOR SA    1.0    USD  6.603  2.2475   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2023-01  2.630320e+08  14.053851      0.811820           1.0  ...   
           2023-02  2.630320e+08  14.512460      0.825355           1.0  ...   
           2023-03  2.630320e+08  14.153832      0.808342           1.0  ...   
           2023-04  2.630320e+08  15.414462      0.795418           1.0  ...   
           2023-05  2.648900e+08  14.840243      0.807298           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                    m_USD_ret  m_GBP_ret  m_local_ret  local_mktval  \
gvkey  iid data_ym                                                    
100001 91  2023-01   0.297352   0.269020     0.297352  1.310951e+09   
           2023-02   0.032632   0.049848     0.032632  1.700765e+09   
           2023-03  -0.024712  -0.044815    -0.024712  1.756265e+09   
           2023-04   0.089066   0.071654     0.089066  1.712864e+09   
           2023-05  -0.037252  -0.022873    -0.037252  1.865423e+09   
...                       ...        ...          ...           ...   
353444 90  2024-03   0.004163   0.004401     0.004163  7.904795e+10   
           2024-04   0.004711   0.013134     0.004711  7.753324e+10   
           2024-05  -0.010551  -0.026026    -0.010551  7.789853e+10   
           2024-06  -0.021327  -0.014667    -0.021327  7.707662e+10   
           2024-07   0.107748   0.090236     0.107748  7.543281e+10   

                      USD_mktval    GBP_mktval  datayear  datayear-1  \
gvkey  iid data_ym                                                     
100001 91  2023-01  1.310951e+09  1.088017e+09      2023        2022   
           2023-02  1.700765e+09  1.380715e+09      2023        2022   
           2023-03  1.756265e+09  1.449542e+09      2023        2022   
           2023-04  1.712864e+09  1.384580e+09      2023        2022   
           2023-05  1.865423e+09  1.483792e+09      2023        2022   
...                          ...           ...       ...         ...   
353444 90  2024-03  7.904795e+10  6.257259e+10      2024        2023   
           2024-04  7.753324e+10  6.138815e+10      2024        2023   
           2024-05  7.789853e+10  6.219443e+10      2024        2023   
           2024-06  7.707662e+10  6.057578e+10      2024        2023   
           2024-07  7.543281e+10  5.968730e+10      2024        2023   

                    m_mktret      beta  
gvkey  iid data_ym                      
100001 91  2023-01  0.070735  0.752327  
           2023-02 -0.029941  1.2

In [1205]:
na_returns.describe()

,ajexm,prccm,trfm,cshom,adjclose,exratd_toGBP,exratd_toUSD,local_ret,USD_fxret,GBP_fxret,...,m_USD_ret,m_GBP_ret,m_local_ret,local_mktval,USD_mktval,GBP_mktval,datayear,datayear-1,m_mktret,beta
count,35239.000000,35239.000000,35239.000000,3.523900e+04,35239.000000,35239.000000,35239.000000,35239.000000,35239.000000,35239.000000,...,35239.000000,35239.000000,35239.000000,3.523900e+04,3.523900e+04,3.523900e+04,35239.000000,35239.000000,35239.000000,35239.000000
mean,1.041720,52.639311,1.729468,1.359365e+09,68.689190,0.779400,0.977627,0.010775,-0.000078,-0.003261,...,0.010797,0.006902,0.010846,2.821164e+10,2.810510e+10,2.246297e+10,2023.361361,2022.361361,0.017010,1.121596
std,1.162036,317.961283,2.956645,7.856098e+09,333.543045,0.059891,0.073103,0.136799,0.004714,0.018198,...,0.136594,0.131873,0.136256,1.400166e+11,1.400195e+11,1.118218e+11,0.480402,0.480402,0.037296,0.899044
min,0.000025,0.000100,1.000000,5.670000e+05,0.000107,0.563793,0.719976,-0.990750,-0.028249,-0.058000,...,-0.756250,-0.754743,-0.756250,2.721000e+03,2.721000e+03,2.150818e+03,2023.000000,2022.000000,-0.045103,-3.640606
25%,1.000000,5.990000,1.000000,5.382400e+07,8.551917,0.785916,1.000000,-0.057768,0.000000,-0.015988,...,-0.058090,-0.056290,-0.057678,5.943862e+08,5.604057e+08,4.490420e+08,2023.000000,2022.000000,-0.026479,0.600861
50%,1.000000,14.699000,1.257500,1.679220e+08,22.935322,0.791264,1.000000,0.004197,0.000000,0.000238,...,0.004008,0.002144,0.004194,2.782245e+09,2.715229e+09,2.172399e+09,2023.000000,2022.000000,0.020874,1.042883
75%,1.000000,37.154500,1.720400,5.578360e+08,58.353104,0.807298,1.000000,0.072154,0.000000,0.008383,...,0.072179,0.063329,0.071964,1.152291e+10,1.147474e+10,9.148568e+09,2024.000000,2023.000000,0.042842,1.527990
max,50.000000,12876.810700,97.982600,2.404173e+11,12876.810700,0.825355,1.000000,1.314186,0.027766,0.054003,...,1.314186,1.263649,1.314186,2.940304e+12,2.940304e+12,2.426794e+12,2024.000000,2023.000000,0.089325,7.924309


## Fundamentals Data

### NA - Loading

In [1234]:
fundamentals_df = pd.read_csv(
    "phase1_fundamentals.csv", 
    parse_dates=["datadate"]
)

fundamentals_df

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,at,ceq,oiadp,revt,costat
0,100080,2021-12-31,2021,INDL,C,D,STD,BAYER AG,USD,136753.182,37554.496,8084.110,50134.455,A
1,100080,2022-12-31,2022,INDL,C,D,STD,BAYER AG,USD,133461.325,41438.343,9863.422,54226.913,A
2,100080,2023-12-31,2023,INDL,C,D,STD,BAYER AG,USD,128506.347,36395.707,7641.253,52655.337,A
3,100091,2021-12-31,2021,INDL,C,D,STD,RENTOKIL INITIAL PLC,USD,5854.000,1712.000,497.000,4004.000,A
4,100091,2022-12-31,2022,INDL,C,D,STD,RENTOKIL INITIAL PLC,USD,14365.000,4939.000,605.000,4475.000,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152,351590,2022-12-31,2022,INDL,C,D,STD,DAIMLER TRUCK HOLDING AG,USD,68366.372,21430.419,3730.979,54322.030,A
4153,351590,2023-12-31,2023,INDL,C,D,STD,DAIMLER TRUCK HOLDING AG,USD,78713.854,23880.987,5579.784,61777.753,A
4154,353444,2021-12-31,2021,INDL,C,D,STD,HALEON PLC,USD,46650.099,35687.306,2816.529,12924.885,A
4155,353444,2022-12-31,2022,INDL,C,D,STD,HALEON PLC,USD,41948.594,19677.222,2913.448,13082.804,A


In [1235]:
fundamentals_df.dtypes

gvkey                int64
datadate    datetime64[ns]
fyear                int64
indfmt              object
consol              object
popsrc              object
datafmt             object
conm                object
curcd               object
at                 float64
ceq                float64
oiadp              float64
revt               float64
costat              object
dtype: object

Null values out of the 4157 rows:

In [1229]:
print("-- Null values:")
fundamentals_df.isnull().sum()

-- Null values:


gvkey         0
datadate      0
fyear         0
indfmt        0
consol        0
popsrc        0
datafmt       0
conm          0
curcd         0
at            8
ceq          23
oiadp       640
revt          8
costat        0
dtype: int64

### NA - Data Preparation

Companies, identified by gvkeys, can have multiple entries per fiscal year in the 
raw downloaded data. 

This is illustrated below - e.g. because of different 
reporting formats (`indfmt`).

In [1236]:
fundamentals_df[
    fundamentals_df["gvkey"] == 105365
]

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,conm,curcd,at,ceq,oiadp,revt,costat
161,105365,2021-12-31,2021,FS,C,D,STD,AMERICAN TOWER CORP,USD,69887.9,5081.2,NaN,9963.4,A
162,105365,2021-12-31,2021,INDL,C,D,STD,AMERICAN TOWER CORP,USD,69887.9,5081.2,3384.0,9356.9,A
163,105365,2022-12-31,2022,INDL,C,D,STD,AMERICAN TOWER CORP,USD,67194.5,5572.4,2930.3,10711.1,A
164,105365,2022-12-31,2022,FS,C,D,STD,AMERICAN TOWER CORP,USD,67194.5,5572.4,NaN,11216.4,A
165,105365,2023-12-31,2023,INDL,C,D,STD,AMERICAN TOWER CORP,USD,66027.6,4198.2,3561.6,11144.2,A
166,105365,2023-12-31,2023,FS,C,D,STD,AMERICAN TOWER CORP,USD,66027.6,4198.2,NaN,11039.1,A


Consolidate information into single gvkey-fiscalyear entries.

In [1237]:
fundamentals_df = fundamentals_df.groupby(
    ["gvkey", "fyear"]
).first()

fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
100080 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100091 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
351590 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
353444 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   

                                  conm curcd          at        ceq     oiadp  \
gvkey  fyear                                                                    
100080 2021                   BAYER AG   USD  136753.182  37554.496  8084.110   
       2022                   BAYER AG   USD  133461.325  41438.343  9863.422   
       2023                   BAYER AG   USD  128506.347  36395.707  7641.253   
100091 2021       RENTOKIL INITIAL PLC   USD    5854.000   1712.000   497.000   
       2022       RENTOKIL INITIAL PLC   USD   14365.000   4939.000   605.000   
...                                ...   ...         ...        ...       ...   
351590 2022   DAIMLER TRUCK HOLDING AG   USD   68366.372  21430.419  3730.979   
       2023   DAIMLER TRUCK HOLDING AG   USD   78713.854  23880.987  5579.784   
353444 2021                 HALEON PLC   USD   46650.099  35687.306  2816.529   
       2022                 HALEON PLC   USD   41948.594  19677.222  2913.448   
       2023                 HALEON PLC   USD   43379.259  21152.723  3181.953   

                   revt costat  
gvkey  fyear                    
100080 2021   50134.455      A  
       2022   54226.913      A  
       2023   52655.337      A  
100091 2021    4004.000      A  
       2022    4475.000      A  
...                 ...    ...  
351590 2022   54322.030      A  
       2023   61777.753      A  
353444 2021   12924.885      A  
       2022   13082.804      A  
       2023   14396.488      A  

[3525 rows x 12 columns]

Null values are reduced.

In [1239]:
print("-- Null values:")
fundamentals_df.isnull().sum()

-- Null values:


datadate     0
indfmt       0
consol       0
popsrc       0
datafmt      0
conm         0
curcd        0
at           8
ceq         23
oiadp        8
revt         8
costat       0
dtype: int64

Drop the rows with null values.

In [1240]:
fundamentals_df = fundamentals_df.dropna()

fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
100080 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100091 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
351590 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
353444 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   

                                  conm curcd          at        ceq     oiadp  \
gvkey  fyear                                                                    
100080 2021                   BAYER AG   USD  136753.182  37554.496  8084.110   
       2022                   BAYER AG   USD  133461.325  41438.343  9863.422   
       2023                   BAYER AG   USD  128506.347  36395.707  7641.253   
100091 2021       RENTOKIL INITIAL PLC   USD    5854.000   1712.000   497.000   
       2022       RENTOKIL INITIAL PLC   USD   14365.000   4939.000   605.000   
...                                ...   ...         ...        ...       ...   
351590 2022   DAIMLER TRUCK HOLDING AG   USD   68366.372  21430.419  3730.979   
       2023   DAIMLER TRUCK HOLDING AG   USD   78713.854  23880.987  5579.784   
353444 2021                 HALEON PLC   USD   46650.099  35687.306  2816.529   
       2022                 HALEON PLC   USD   41948.594  19677.222  2913.448   
       2023                 HALEON PLC   USD   43379.259  21152.723  3181.953   

                   revt costat  
gvkey  fyear                    
100080 2021   50134.455      A  
       2022   54226.913      A  
       2023   52655.337      A  
100091 2021    4004.000      A  
       2022    4475.000      A  
...                 ...    ...  
351590 2022   54322.030      A  
       2023   61777.753      A  
353444 2021   12924.885      A  
       2022   13082.804      A  
       2023   14396.488      A  

[3502 rows x 12 columns]

Join with the exchange rates to USD in order to standardise the values to USD.
- Last for balance sheet
- Average for income statement

In [1261]:
m_exrts_flat = m_exrts.reset_index()
m_exrts_flat["datayear"] = m_exrts_flat["data_ym"].dt.year

m_exrts_flat

,curd,data_ym,exratd_toGBP,exratd_toUSD,datayear
0,AED,2021-01,0.198491,0.272251,2021
1,AED,2021-02,0.195503,0.272258,2021
2,AED,2021-03,0.197625,0.272248,2021
3,AED,2021-04,0.197083,0.272251,2021
4,AED,2021-05,0.191637,0.272259,2021
...,...,...,...,...,...
6752,ZWL,2024-04,0.002350,0.002944,2024
6753,ZWL,2024-05,0.002350,0.002991,2024
6754,ZWL,2024-06,0.002350,0.002970,2024
6755,ZWL,2024-07,0.002350,0.003018,2024


In [1272]:
y_bs_exrt = m_exrts_flat.drop(columns="data_ym").groupby(
    ["curd", "datayear"]
).last()[["exratd_toUSD"]].rename(
    columns={"exratd_toUSD": "bs_toUSD"}
)

y_bs_exrt

bs_toUSD
curd datayear          
AED  2021      0.272263
     2022      0.272257
     2023      0.272249
     2024      0.272248
AFN  2021      0.009641
...                 ...
ZMW  2024      0.038023
ZWL  2021      0.003183
     2022      0.002832
     2023      0.002994
     2024      0.003032

[615 rows x 1 columns]

In [1273]:
y_is_exrt = m_exrts_flat.drop(columns="data_ym").groupby(
    ["curd", "datayear"]
).mean()[["exratd_toUSD"]].rename(columns={"exratd_toUSD": "is_toUSD"})

y_is_exrt

is_toUSD
curd datayear          
AED  2021      0.272250
     2022      0.272247
     2023      0.272256
     2024      0.272254
AFN  2021      0.011940
...                 ...
ZMW  2024      0.039096
ZWL  2021      0.003230
     2022      0.002893
     2023      0.002933
     2024      0.002985

[615 rows x 1 columns]

In [1274]:
y_fs_exrt = pd.concat(
    [y_bs_exrt, y_is_exrt], 
    axis=1
)

y_fs_exrt

bs_toUSD  is_toUSD
curd datayear                    
AED  2021      0.272263  0.272250
     2022      0.272257  0.272247
     2023      0.272249  0.272256
     2024      0.272248  0.272254
AFN  2021      0.009641  0.011940
...                 ...       ...
ZMW  2024      0.038023  0.039096
ZWL  2021      0.003183  0.003230
     2022      0.002832  0.002893
     2023      0.002994  0.002933
     2024      0.003032  0.002985

[615 rows x 2 columns]

In [1276]:
fundamentals_df = fundamentals_df.reset_index().merge(
    y_fs_exrt, 
    how="left", 
    left_on=["curcd", "fyear"], 
    right_index=True, 
).set_index(
    ["gvkey", "fyear"]
)

fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
100080 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100091 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
351590 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
353444 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   

                                  conm curcd          at        ceq     oiadp  \
gvkey  fyear                                                                    
100080 2021                   BAYER AG   USD  136753.182  37554.496  8084.110   
       2022                   BAYER AG   USD  133461.325  41438.343  9863.422   
       2023                   BAYER AG   USD  128506.347  36395.707  7641.253   
100091 2021       RENTOKIL INITIAL PLC   USD    5854.000   1712.000   497.000   
       2022       RENTOKIL INITIAL PLC   USD   14365.000   4939.000   605.000   
...                                ...   ...         ...        ...       ...   
351590 2022   DAIMLER TRUCK HOLDING AG   USD   68366.372  21430.419  3730.979   
       2023   DAIMLER TRUCK HOLDING AG   USD   78713.854  23880.987  5579.784   
353444 2021                 HALEON PLC   USD   46650.099  35687.306  2816.529   
       2022                 HALEON PLC   USD   41948.594  19677.222  2913.448   
       2023                 HALEON PLC   USD   43379.259  21152.723  3181.953   

                   revt costat  bs_toUSD  is_toUSD  
gvkey  fyear                                        
100080 2021   50134.455      A       1.0       1.0  
       2022   54226.913      A       1.0       1.0  
       2023   52655.337      A       1.0       1.0  
100091 2021    4004.000      A       1.0       1.0  
       2022    4475.000      A       1.0       1.0  
...                 ...    ...       ...       ...  
351590 2022   54322.030      A       1.0       1.0  
       2023   61777.753      A       1.0       1.0  
353444 2021   12924.885      A       1.0       1.0  
       2022   13082.804      A       1.0       1.0  
       2023   14396.488      A       1.0       1.0  

[3502 rows x 14 columns]

Apply the exchange rates.

In [1280]:
fundamentals_df[
    fundamentals_df["curcd"] == "CAD"
]

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
105239 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
105517 2021  2021-12-31   INDL      C      D     STD   
105576 2021  2021-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
311798 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
326502 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   

                                    conm curcd         at       ceq     oiadp  \
gvkey  fyear                                                                    
105239 2021            ARC RESOURCES LTD   CAD  11380.300  5927.500  1124.600   
       2022            ARC RESOURCES LTD   CAD  11623.900  6653.500  3037.000   
       2023            ARC RESOURCES LTD   CAD  12382.900  7427.800  2068.500   
105517 2021   BADGER INFRA SOLUTIONS LTD   CAD    606.406   281.202    -4.910   
105576 2021     DIVERSIFIED ROYALTY CORP   CAD    380.764   191.525    33.106   
...                                  ...   ...        ...       ...       ...   
311798 2021        SUNSHINE OILSANDS LTD   CAD    755.724   177.050   -18.902   
       2022        SUNSHINE OILSANDS LTD   CAD    747.719   111.008   -30.982   
       2023        SUNSHINE OILSANDS LTD   CAD    745.932    92.265   -32.129   
326502 2021            VIVO CANNABIS INC   CAD    118.414    86.029   -19.095   
       2022            VIVO CANNABIS INC   CAD     45.420    37.972   -13.436   

                  revt costat  bs_toUSD  is_toUSD  
gvkey  fyear                                       
105239 2021   4084.800      A  0.790254  0.797758  
       2022   7662.300      A  0.738885  0.767227  
       2023   6021.800      A  0.757223  0.742049  
105517 2021    568.752      A  0.790254  0.797758  
105576 2021     37.281      A  0.790254  0.797758  
...                ...    ...       ...       ...  
311798 2021      0.143      A  0.790254  0.797758  
       2022     14.667      A  0.738885  0.767227  
       2023     29.555      A  0.757223  0.742049  
326502 2021     24.993      I  0.790254  0.797758  
       2022     25.412      I  0.738885  0.767227  

[324 rows x 14 columns]

In [1281]:
fundamentals_df["at"] *= fundamentals_df["bs_toUSD"]
fundamentals_df["ceq"] *= fundamentals_df["bs_toUSD"]

fundamentals_df["oiadp"] *= fundamentals_df["is_toUSD"]
fundamentals_df["revt"] *= fundamentals_df["is_toUSD"]

In [1282]:
fundamentals_df[
    fundamentals_df["curcd"] == "CAD"
]

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
105239 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
105517 2021  2021-12-31   INDL      C      D     STD   
105576 2021  2021-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
311798 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
326502 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   

                                    conm curcd           at          ceq  \
gvkey  fyear                                                               
105239 2021            ARC RESOURCES LTD   CAD  8993.326076  4684.229793   
       2022            ARC RESOURCES LTD   CAD  8588.726995  4916.172288   
       2023            ARC RESOURCES LTD   CAD  9376.612781  5624.498657   
105517 2021   BADGER INFRA SOLUTIONS LTD   CAD   479.214686   222.220968   
105576 2021     DIVERSIFIED ROYALTY CORP   CAD   300.900223   151.353372   
...                                  ...   ...          ...          ...   
311798 2021        SUNSHINE OILSANDS LTD   CAD   597.213813   139.914447   
       2022        SUNSHINE OILSANDS LTD   CAD   552.478459    82.022162   
       2023        SUNSHINE OILSANDS LTD   CAD   564.836632    69.865151   
326502 2021            VIVO CANNABIS INC   CAD    93.577121    67.984750   
       2022            VIVO CANNABIS INC   CAD    33.560163    28.056947   

                    oiadp         revt costat  bs_toUSD  is_toUSD  
gvkey  fyear                                                       
105239 2021    897.159030  3258.683270      A  0.790254  0.797758  
       2022   2330.068252  5878.723072      A  0.738885  0.767227  
       2023   1534.928971  4468.472457      A  0.757223  0.742049  
105517 2021     -3.916993   453.726652      A  0.790254  0.797758  
105576 2021     26.410588    29.741229      A  0.790254  0.797758  
...                   ...          ...    ...       ...       ...  
311798 2021    -15.079228     0.114079      A  0.790254  0.797758  
       2022    -23.770225    11.252918      A  0.738885  0.767227  
       2023    -23.841302    21.931267      A  0.757223  0.742049  
326502 2021    -15.233196    19.938374      I  0.790254  0.797758  
       2022    -10.308461    19.496771      I  0.738885  0.767227  

[324 rows x 14 columns]

Compute investment by percentage change in assets.

In [1284]:
fundamentals_df["investment"] = fundamentals_df.groupby(
    level="gvkey"
)["at"].pct_change()

fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
100080 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100091 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
351590 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
353444 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   

                                  conm curcd          at        ceq     oiadp  \
gvkey  fyear                                                                    
100080 2021                   BAYER AG   USD  136753.182  37554.496  8084.110   
       2022                   BAYER AG   USD  133461.325  41438.343  9863.422   
       2023                   BAYER AG   USD  128506.347  36395.707  7641.253   
100091 2021       RENTOKIL INITIAL PLC   USD    5854.000   1712.000   497.000   
       2022       RENTOKIL INITIAL PLC   USD   14365.000   4939.000   605.000   
...                                ...   ...         ...        ...       ...   
351590 2022   DAIMLER TRUCK HOLDING AG   USD   68366.372  21430.419  3730.979   
       2023   DAIMLER TRUCK HOLDING AG   USD   78713.854  23880.987  5579.784   
353444 2021                 HALEON PLC   USD   46650.099  35687.306  2816.529   
       2022                 HALEON PLC   USD   41948.594  19677.222  2913.448   
       2023                 HALEON PLC   USD   43379.259  21152.723  3181.953   

                   revt costat  bs_toUSD  is_toUSD  investment  
gvkey  fyear                                                    
100080 2021   50134.455      A       1.0       1.0         NaN  
       2022   54226.913      A       1.0       1.0   -0.024072  
       2023   52655.337      A       1.0       1.0   -0.037127  
100091 2021    4004.000      A       1.0       1.0         NaN  
       2022    4475.000      A       1.0       1.0    1.453878  
...                 ...    ...       ...       ...         ...  
351590 2022   54322.030      A       1.0       1.0    0.096925  
       2023   61777.753      A       1.0       1.0    0.151353  
353444 2021   12924.885      A       1.0       1.0         NaN  
       2022   13082.804      A       1.0       1.0   -0.100782  
       2023   14396.488      A       1.0       1.0    0.034105  

[3502 rows x 15 columns]

Compute operating profitability.

In [1285]:
fundamentals_df["opm"] = fundamentals_df["oiadp"] / fundamentals_df["revt"]

fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
100080 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100091 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
351590 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
353444 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   

                                  conm curcd          at        ceq     oiadp  \
gvkey  fyear                                                                    
100080 2021                   BAYER AG   USD  136753.182  37554.496  8084.110   
       2022                   BAYER AG   USD  133461.325  41438.343  9863.422   
       2023                   BAYER AG   USD  128506.347  36395.707  7641.253   
100091 2021       RENTOKIL INITIAL PLC   USD    5854.000   1712.000   497.000   
       2022       RENTOKIL INITIAL PLC   USD   14365.000   4939.000   605.000   
...                                ...   ...         ...        ...       ...   
351590 2022   DAIMLER TRUCK HOLDING AG   USD   68366.372  21430.419  3730.979   
       2023   DAIMLER TRUCK HOLDING AG   USD   78713.854  23880.987  5579.784   
353444 2021                 HALEON PLC   USD   46650.099  35687.306  2816.529   
       2022                 HALEON PLC   USD   41948.594  19677.222  2913.448   
       2023                 HALEON PLC   USD   43379.259  21152.723  3181.953   

                   revt costat  bs_toUSD  is_toUSD  investment       opm  
gvkey  fyear                                                              
100080 2021   50134.455      A       1.0       1.0         NaN  0.161249  
       2022   54226.913      A       1.0       1.0   -0.024072  0.181892  
       2023   52655.337      A       1.0       1.0   -0.037127  0.145118  
100091 2021    4004.000      A       1.0       1.0         NaN  0.124126  
       2022    4475.000      A       1.0       1.0    1.453878  0.135196  
...                 ...    ...       ...       ...         ...       ...  
351590 2022   54322.030      A       1.0       1.0    0.096925  0.068683  
       2023   61777.753      A       1.0       1.0    0.151353  0.090320  
353444 2021   12924.885      A       1.0       1.0         NaN  0.217915  
       2022   13082.804      A       1.0       1.0   -0.100782  0.222693  
       2023   14396.488      A       1.0       1.0    0.034105  0.221023  

[3502 rows x 16 columns]

Finally, drop null rows so that the table is ready for linking.

In [1297]:
fundamentals_df = fundamentals_df.dropna()

fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
100080 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100091 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100095 2022  2022-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
351491 2023  2023-12-31   INDL      C      D     STD   
351590 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
353444 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   

                                  conm curcd          at        ceq     oiadp  \
gvkey  fyear                                                                    
100080 2022                   BAYER AG   USD  133461.325  41438.343  9863.422   
       2023                   BAYER AG   USD  128506.347  36395.707  7641.253   
100091 2022       RENTOKIL INITIAL PLC   USD   14365.000   4939.000   605.000   
       2023       RENTOKIL INITIAL PLC   USD   14174.000   5209.000   971.000   
100095 2022                  BUNZL PLC   USD   10442.627   3278.412   920.785   
...                                ...   ...         ...        ...       ...   
351491 2023            IVECO GROUP N V   USD   20321.775   2600.878  1158.402   
351590 2022   DAIMLER TRUCK HOLDING AG   USD   68366.372  21430.419  3730.979   
       2023   DAIMLER TRUCK HOLDING AG   USD   78713.854  23880.987  5579.784   
353444 2022                 HALEON PLC   USD   41948.594  19677.222  2913.448   
       2023                 HALEON PLC   USD   43379.259  21152.723  3181.953   

                   revt costat  bs_toUSD  is_toUSD  investment       opm  
gvkey  fyear                                                              
100080 2022   54226.913      A       1.0       1.0   -0.024072  0.181892  
       2023   52655.337      A       1.0       1.0   -0.037127  0.145118  
100091 2022    4475.000      A       1.0       1.0    1.453878  0.135196  
       2023    6847.000      A       1.0       1.0   -0.013296  0.141814  
100095 2022   14506.394      A       1.0       1.0    0.084024  0.063474  
...                 ...    ...       ...       ...         ...       ...  
351491 2023   17920.964      A       1.0       1.0    0.187452  0.064639  
351590 2022   54322.030      A       1.0       1.0    0.096925  0.068683  
       2023   61777.753      A       1.0       1.0    0.151353  0.090320  
353444 2022   13082.804      A       1.0       1.0   -0.100782  0.222693  
       2023   14396.488      A       1.0       1.0    0.034105  0.221023  

[2298 rows x 16 columns]

## Linking

In [1243]:
emissions_df

,institutionid,fiscalyear,periodenddate,di_319413,di_319414,di_319415,companyid,gvkey,companyname,country,periodend_ym
0,13959,2022,2022-12-31,18853.165084,72589.700681,1.323541e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2022-12
1,13959,2023,2023-12-31,23250.617734,89521.063147,1.632254e+06,246652,122594,State Farm Mutual Automobile Insurance Company,United States,2023-12
2,14193,2022,2022-12-31,16736.521444,51330.155513,7.097935e+05,417155,122554,United Services Automobile Association,United States,2022-12
3,14193,2023,2023-12-31,19585.900250,60069.071644,8.306352e+05,417155,122554,United Services Automobile Association,United States,2023-12
4,14467,2022,2022-12-31,799.396226,2451.712127,3.390228e+04,675974,263562,Everlake Life Insurance Company,United States,2022-12
...,...,...,...,...,...,...,...,...,...,...,...
26402,112329550,2022,2022-10-31,489.910950,1022.660743,6.379460e+03,1849817514,361808,Kawan Renergy Berhad,Malaysia,2022-10
26403,113511623,2022,2022-12-31,291.908186,183.331947,5.197957e+02,1855364409,358709,ELSA Solutions S.p.A.,Italy,2022-12
26404,113511665,2022,2022-09-30,12.539324,0.790235,6.400957e+00,1855399529,358653,"SEIYU KOGYO Co.,Ltd.",Japan,2022-09
26405,114230580,2022,2022-03-31,2.811856,0.177205,1.435370e+00,1859487646,359029,KET Inc.,Japan,2022-03


In [1247]:
na_returns

datadate        conm  ajexm curcdm  prccm    trfm  \
gvkey  iid data_ym                                                       
100001 91  2023-01 2023-01-31    ACCOR SA    1.0    USD  6.466  2.1735   
           2023-02 2023-02-28    ACCOR SA    1.0    USD  6.677  2.1735   
           2023-03 2023-03-31    ACCOR SA    1.0    USD  6.512  2.1735   
           2023-04 2023-04-30    ACCOR SA    1.0    USD  7.092  2.1735   
           2023-05 2023-05-31    ACCOR SA    1.0    USD  6.603  2.2475   
...                       ...         ...    ...    ...    ...     ...   
353444 90  2024-03 2024-03-31  HALEON PLC    1.0    USD  8.490  1.0253   
           2024-04 2024-04-30  HALEON PLC    1.0    USD  8.530  1.0253   
           2024-05 2024-05-31  HALEON PLC    1.0    USD  8.440  1.0253   
           2024-06 2024-06-30  HALEON PLC    1.0    USD  8.260  1.0253   
           2024-07 2024-07-31  HALEON PLC    1.0    USD  9.150  1.0253   

                           cshom   adjclose  exratd_toGBP  exratd_toUSD  ...  \
gvkey  iid data_ym                                                       ...   
100001 91  2023-01  2.630320e+08  14.053851      0.811820           1.0  ...   
           2023-02  2.630320e+08  14.512460      0.825355           1.0  ...   
           2023-03  2.630320e+08  14.153832      0.808342           1.0  ...   
           2023-04  2.630320e+08  15.414462      0.795418           1.0  ...   
           2023-05  2.648900e+08  14.840243      0.807298           1.0  ...   
...                          ...        ...           ...           ...  ...   
353444 90  2024-03  9.132301e+09   8.704797      0.791766           1.0  ...   
           2024-04  9.132301e+09   8.745809      0.798403           1.0  ...   
           2024-05  9.132301e+09   8.653532      0.785916           1.0  ...   
           2024-06  9.132301e+09   8.468978      0.791264           1.0  ...   
           2024-07  9.132301e+09   9.381495      0.778756           1.0  ...   

                    m_USD_ret  m_GBP_ret  m_local_ret  local_mktval  \
gvkey  iid data_ym                                                    
100001 91  2023-01   0.297352   0.269020     0.297352  1.310951e+09   
           2023-02   0.032632   0.049848     0.032632  1.700765e+09   
           2023-03  -0.024712  -0.044815    -0.024712  1.756265e+09   
           2023-04   0.089066   0.071654     0.089066  1.712864e+09   
           2023-05  -0.037252  -0.022873    -0.037252  1.865423e+09   
...                       ...        ...          ...           ...   
353444 90  2024-03   0.004163   0.004401     0.004163  7.904795e+10   
           2024-04   0.004711   0.013134     0.004711  7.753324e+10   
           2024-05  -0.010551  -0.026026    -0.010551  7.789853e+10   
           2024-06  -0.021327  -0.014667    -0.021327  7.707662e+10   
           2024-07   0.107748   0.090236     0.107748  7.543281e+10   

                      USD_mktval    GBP_mktval  datayear  datayear-1  \
gvkey  iid data_ym                                                     
100001 91  2023-01  1.310951e+09  1.088017e+09      2023        2022   
           2023-02  1.700765e+09  1.380715e+09      2023        2022   
           2023-03  1.756265e+09  1.449542e+09      2023        2022   
           2023-04  1.712864e+09  1.384580e+09      2023        2022   
           2023-05  1.865423e+09  1.483792e+09      2023        2022   
...                          ...           ...       ...         ...   
353444 90  2024-03  7.904795e+10  6.257259e+10      2024        2023   
           2024-04  7.753324e+10  6.138815e+10      2024        2023   
           2024-05  7.789853e+10  6.219443e+10      2024        2023   
           2024-06  7.707662e+10  6.057578e+10      2024        2023   
           2024-07  7.543281e+10  5.968730e+10      2024        2023   

                    m_mktret      beta  
gvkey  iid data_ym                      
100001 91  2023-01  0.070735  0.752327  
           2023-02 -0.029941  1.2

In [1248]:
fundamentals_df

datadate indfmt consol popsrc datafmt  \
gvkey  fyear                                           
100080 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
100091 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
...                 ...    ...    ...    ...     ...   
351590 2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   
353444 2021  2021-12-31   INDL      C      D     STD   
       2022  2022-12-31   INDL      C      D     STD   
       2023  2023-12-31   INDL      C      D     STD   

                                  conm curcd          at        ceq     oiadp  \
gvkey  fyear                                                                    
100080 2021                   BAYER AG   USD  136753.182  37554.496  8084.110   
       2022                   BAYER AG   USD  133461.325  41438.343  9863.422   
       2023                   BAYER AG   USD  128506.347  36395.707  7641.253   
100091 2021       RENTOKIL INITIAL PLC   USD    5854.000   1712.000   497.000   
       2022       RENTOKIL INITIAL PLC   USD   14365.000   4939.000   605.000   
...                                ...   ...         ...        ...       ...   
351590 2022   DAIMLER TRUCK HOLDING AG   USD   68366.372  21430.419  3730.979   
       2023   DAIMLER TRUCK HOLDING AG   USD   78713.854  23880.987  5579.784   
353444 2021                 HALEON PLC   USD   46650.099  35687.306  2816.529   
       2022                 HALEON PLC   USD   41948.594  19677.222  2913.448   
       2023                 HALEON PLC   USD   43379.259  21152.723  3181.953   

                   revt costat  
gvkey  fyear                    
100080 2021   50134.455      A  
       2022   54226.913      A  
       2023   52655.337      A  
100091 2021    4004.000      A  
       2022    4475.000      A  
...                 ...    ...  
351590 2022   54322.030      A  
       2023   61777.753      A  
353444 2021   12924.885      A  
       2022   13082.804      A  
       2023   14396.488      A  

[3502 rows x 12 columns]

## Checks on Duplicates

In [ ]:
""" Now determine the number of unique companies in the 
different years
"""
print(f"All years: {np.unique(df['fiscalyear'])}")

for year in np.unique(df['fiscalyear']):
    print(f"{year} => {np.unique(df[df['fiscalyear'] == year]['gvkey']).size}")

NameError: name 'df' is not defined

In [ ]:
"""
All years: [2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016 2017 2018 2019 2020 2021 2022 2023]
2002 => 1763
2003 => 2000
2004 => 2885
2005 => 3880
2006 => 4170
2007 => 4307
2008 => 4269
2009 => 4563
2010 => 4723
2011 => 4833
2012 => 4868
2013 => 5757
2014 => 6154
2015 => 6235
2016 => 13882
2017 => 14786
2018 => 16979
2019 => 17378
2020 => 23353
2021 => 23386
2022 => 24053
2023 => 9725
"""

In [ ]:
# unique non-nan in returns
returns_gvkeys = np.unique(returns_df[~np.isnan(returns_df["gvkey"])]["gvkey"])

print(returns_gvkeys.size)

24110


In [ ]:
env_ret_common_gvkeys = np.intersect1d(
    env_gvkeys, 
    returns_gvkeys, 
    assume_unique=True
)

print(env_ret_common_gvkeys.size)

24110


In [ ]:
""" Check the missing gvkeys to see what country they are from """
missing_gvkeys = np.setdiff1d(env_gvkeys, env_ret_common_gvkeys, assume_unique=True)

print(missing_gvkeys.size)

7400


In [ ]:
""" Create representatives and break down their distribution """
df_missing_reprs_idx = [(df["gvkey"] == gvkey).idxmax() for gvkey in missing_gvkeys]
missing_dist = df.iloc[df_missing_reprs_idx][["gvkey", "country"]].groupby(
    "country"
).count().reset_index().sort_values('gvkey', ascending=False)

In [ ]:
missing_dist

,country,gvkey
79,United States,5220
10,Canada,803
78,United Kingdom,271
13,China,113
24,France,111
...,...,...
58,Peru,1
57,Panama,1
56,Pakistan,1
48,Marshall Islands,1


In [ ]:
missing_dist[missing_dist["country"] != "United States"]["gvkey"].sum()

2180

In [15]:
company_dups = df[df.duplicated('gvkey', keep=False) == True].sort_values(by="gvkey")

In [26]:
last_iid = None
last_fyears = set()
for i, row in company_dups.iterrows():
    current_iid = row["institutionid"]
    current_fyear = row["fiscalyear"]
    if last_iid is not None and current_iid == last_iid:
        if current_fyear in last_fyears:
            raise Exception(f"Fyear clash!, iid: {current_iid}, prev_fyears: {last_fyears}, clash: {current_fyear}")
        else:
            last_fyears.add(current_fyear)
    else:
        last_iid = current_iid
        last_fyears.clear()
        last_fyears.add(current_fyear)

print("No issues...")

Exception: Fyear clash!, iid: 4415462, prev_fyears: {2022}, clash: 2022

In [31]:
company_dups.head(20)

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
5,30D218CF-2E2A-46B4-AF72-CADF593290E8,4074603,USD,713A00,2022,01/01/2023,0.285,0.021,2.079,0.153,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
2503,1989BCA5-218F-4857-A3CE-1F2114D67F8E,4074603,USD,713A00,2023,31/12/2023,0.302,0.021,2.098,0.147,...,NaN,NaN,76011,1961.0,NaN,NaN,972 595 5000,NaN,NaN,www.sixflags.com
4988,5B249039-D1E2-43D2-825B-DDD233FE2FE0,4996548,USD,561300,2023,31/12/2023,0.015,0.005,0.125,0.040,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
40,5F205052-5493-41C8-9C8C-1E0FE1FC5DC0,4996548,USD,561300,2022,01/01/2023,0.014,0.005,0.122,0.041,...,NaN,NaN,75024,2007.0,NaN,NaN,972 692 2400,NaN,NaN,bgsf.com
1,01C53196-7DD9-42C7-9D3B-F152BFB3A364,4054841,USD,445000A,2022,01/01/2023,3.061,0.003,186.279,0.203,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
2462,793C84DE-3E9D-4CE0-8768-647BA85F9272,4054841,USD,445000A,2023,31/12/2023,3.183,0.003,181.084,0.189,...,NaN,NaN,1506 MA,1867.0,NaN,NaN,31 88 659 9111,NaN,NaN,www.aholddelhaize.com
110,A706AF20-7252-4C95-AA58-2481363942C5,10175068,USD,722000,2022,02/01/2023,0.095,0.053,0.696,0.389,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
6071,96731D0E-B576-41E4-8386-DAD27811DF60,10175068,USD,722000,2023,01/01/2024,0.092,0.054,0.625,0.368,...,NaN,NaN,33309,2011.0,NaN,NaN,954-618-2000,NaN,NaN,www.burgerfi.com
99,0446986D-7007-4B72-AA5B-49AA5ADB4CF2,28295169,USD,541512,2022,01/01/2023,0.042,0.020,0.362,0.170,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com
6011,9AABC54A-2E82-4C6A-99E2-489EF08AE342,28295169,USD,541512,2023,31/12/2023,0.051,0.018,0.420,0.147,...,NaN,NaN,55425,2016.0,NaN,NaN,952 851 5200,NaN,NaN,www.skywatertechnology.com


In [32]:
company_dups[company_dups["institutionid"] == 4415462]

,periodid,institutionid,reportedcurrencyisocode,tcprimarysectorid,fiscalyear,periodenddate,di_319380,di_319381,di_319382,di_319383,...,streetaddress3,streetaddress4,zipcode,yearfounded,monthfounded,dayfounded,officephonevalue,otherphonevalue,officefaxvalue,webpage
23,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,050059,1993.0,NaN,NaN,7 727 244 5484,NaN,7 727 244 5480,www.homecredit.kz
24,5282F3CE-9CFA-4830-9116-3B36EFDE5089,4415462,USD,52A000,2022,01/01/2023,0.001,0.001,0.054,0.037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


array(['United States', 'Netherlands', 'United Kingdom', 'Canada',
       'Kazakhstan', 'Belarus', 'Australia', 'Belgium', 'Austria',
       'Finland', 'Ireland', nan, 'Singapore', 'France', 'Denmark',
       'Japan', 'Israel', 'Italy', 'South Africa', 'Thailand', 'Germany',
       'China', 'Hong Kong', 'Luxembourg', 'India', 'Switzerland',
       'Malaysia', 'South Korea', 'Kenya', 'New Zealand', 'Spain',
       'Pakistan', 'Saudi Arabia', 'Sweden', 'British Virgin Islands',
       'Kuwait', 'Turkey', 'Philippines', 'Mauritius', 'Bangladesh',
       'Cayman Islands', 'Botswana', 'Egypt', 'Malta', 'Malawi',
       'Jamaica', 'Bermuda', 'Colombia', 'Mexico', 'Norway', 'Brazil',
       'Bahrain', 'Morocco', 'Indonesia', 'Romania', 'Russia',
       'Ivory Coast', 'Tunisia', 'Greece', 'Vietnam', 'Taiwan', 'Nigeria',
       'Oman', 'Qatar', 'Portugal', 'United Arab Emirates', 'Jersey',
       'Poland', 'Bulgaria', 'Chile', 'Reunion', 'Ghana', 'Monaco',
       'Bahamas', 'Guernsey'], dtype=o